# Extract text from 10K files
We will extract items: item_1, item_1A, and item_7A from company 10K files. Since we will be passing this text into an LLM, to save on computational resources we identify the top two most important paragraphs - identified by the count of keywords in them - in each item and extract these

In [1]:
import re
import pandas as pd

from llm.utils.utilities import item_search, clean_text

keywords = ["risk", "financial", "performance", "dividend", "buyback", "operations", "market", "legal", "increase", "decrease"]
sequence_length = 2

In [4]:
content = """
<Header>
<FileStats>
    <FileName>20230103_10-K_edgar_data_1487931_0001477932-23-000012.txt</FileName>
    <GrossFileSize>5742966</GrossFileSize>
    <NetFileSize>212466</NetFileSize>
    <NonText_DocumentType_Chars>1067742</NonText_DocumentType_Chars>
    <HTML_Chars>1315185</HTML_Chars>
    <XBRL_Chars>1242285</XBRL_Chars>
    <XML_Chars>1772129</XML_Chars>
    <N_Exhibits>10</N_Exhibits>
</FileStats>
<SEC-Header>
0001477932-23-000012.hdr.sgml : 20230103
<ACCEPTANCE-DATETIME>20230103120448
ACCESSION NUMBER:		0001477932-23-000012
CONFORMED SUBMISSION TYPE:	10-K
PUBLIC DOCUMENT COUNT:		87
CONFORMED PERIOD OF REPORT:	20211231
FILED AS OF DATE:		20230103
DATE AS OF CHANGE:		20230103

FILER:

	COMPANY DATA:	
		COMPANY CONFORMED NAME:			FISION Corp
		CENTRAL INDEX KEY:			0001487931
		STANDARD INDUSTRIAL CLASSIFICATION:	SERVICES-COMPUTER PROCESSING & DATA PREPARATION [7374]
		IRS NUMBER:				272205792
		FISCAL YEAR END:			1231

	FILING VALUES:
		FORM TYPE:		10-K
		SEC ACT:		1934 Act
		SEC FILE NUMBER:	000-53929
		FILM NUMBER:		23501097

	BUSINESS ADDRESS:	
		STREET 1:		1650 WEST END BOULEVARD
		STREET 2:		SUITE 100
		CITY:			MINNEAPOLIS
		STATE:			MN
		ZIP:			55416
		BUSINESS PHONE:		763-259-5825

	MAIL ADDRESS:	
		STREET 1:		1650 WEST END BOULEVARD
		STREET 2:		SUITE 100
		CITY:			MINNEAPOLIS
		STATE:			MN
		ZIP:			55416

	FORMER COMPANY:	
		FORMER CONFORMED NAME:	DE Acquisition 6, Inc.
		DATE OF NAME CHANGE:	20100324

</SEC-Header>
</Header>

 0001477932-23-000012.txt : 20230103

10-K
 1
 fssn_10k.htm
 FORM 10-K

fssn_10k.htm 
 UNITED STATES SECURITIES AND EXCHANGE COMMISSION Washington, D.C. 20549 FORM (Mark One) ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(D) OF THE SECURITIES EXCHANGE ACT OF 1934 For the fiscal year ended Or TRANSITION REPORT UNDER SECTION 13 OR 15(D) OF THE SECURITIES EXCHANGE ACT OF 1934 For the transition period from ________ to _________ Commission File No. (Exact name of registrant as specified in its charter) (State or other jurisdiction of incorporation) (IRS Employer Identification No.) , , (Address of principal executive offices) (Zip Code) ) Registrant s telephone number, including area code Securities registered pursuant to Section 12(b) of the Act: None Securities registered pursuant to Section 12(g) of the Act: Common Stock, .0001 par value per share Indicate by check mark if the registrant is a well-known seasoned issuer, as defined in Rule 405 of the Securities Act Yes Indicate by check mark if the registrant is not required to file reports pursuant to Section 13 or 15(d) of the Act Yes Indicate by check mark whether the registrant (1) has filed all reports required to be filed by Section 13 or 15(d) of the Securities Exchange Act of 1934 during the period 12 months (or for such shorter period that the registrant was required to file such reports), and (2) has been subject to the filing requirements for the past 90 days No Indicate by check mark whether the registrant has submitted electronically and posted on its corporate Website, if any, every Interactive Data File required to be submitted and posted pursuant to Rule 405 of Regulation S-T 232.405 of this chapter) during the preceding 12 months (or for such shorter period that the registrant was required to submit and post such files). No Indicate by check mark if disclosure of delinquent filers pursuant to Item 405 of Regulation S-K is not contained herein, and will not be contained, to the best of registrant s knowledge, in definitive proxy or information statements incorporated by reference in Part III of this Form 10-K or any amendment to this Form 10-K. Indicate by check mark whether the registrant is a large accelerated filed, an accelerated filer, a non-accelerated filer or a smaller reporting company: Large accelerated filer Accelerated filer Smaller reporting company Emerging growth company If an emerging growth company, indicated by check mark if the registrant has elected not to use the extended transition period for complying with any new or revised financial accounting standards provided pursuant to Section 13(a) of the Exchange Act. Indicate by check mark whether the registrant is a shell company (as defined in Rule 12b-2 of the act). Yes No State the aggregate market value of the voting and non-voting common equity held by non-affiliates computed by reference to the price at which the common stock was last sold as of the last business day of the registrant s most recently completed second fiscal quarter: on June 30, 2022. Indicate the number of the registrant s shares of common stock outstanding, as of the latest practicable date: shares of common stock are outstanding as of December 22, 2022. DOCUMENTS INCORPORATED BY REFERENCE: None Table of Contents Part I Page No. Item 1 Business 4 Item 1A Risk factors 8 Item 1B Unresolved Staff Comments 14 Item 2 Properties 15 Item 3 Legal Proceedings 15 Item 4 Mine Safety Disclosures 15 Part II Item 5 Market for Registrant s Common Equity, Related Stockholder Matters and Issuer Purchases of Equity Securities 16 Item 6 Selected Financial Data 18 Item 7 Management s Discussion and Analysis of Financial Condition and Results of Operations 18 Item 7A Quantitative and Qualitative Disclosures About Market Risk 25 Item 8 Financial Statements and Supplementary Data 25 Item 9 Changes in and Disagreements With Accountants on Accounting and Financial Disclosure 25 Item 9A Controls and Procedures 25 Item 9B Other Information 26 Part III Item 10 Directors, Executive Officers and Corporate Governance 27 Item 11 Executive Compensation 28 Item 12 Security Ownership of Certain Beneficial Owners and Management and Related Stockholder Matters 30 Item 13 Certain Relationships and Related Transactions, and Director Independence 31 Item 14 Principal Accounting Fees and Services 32 Part IV Item 15 Exhibits, Financial Statement Schedules 33 Signatures 35 2 Table of Contents Corporate Contact Information Our current principal executive and administrative offices are located at 1650 West End Boulevard, Suite 100, Minneapolis, Minnesota 55416; our telephone number is (763) 259-5825 and we maintain a website atwww.fisiononline.com. Special Note Regarding Forward-Looking Statements There are many statements in this Annual Report on Form 10-K that are not historical facts. These forward-looking statements can be identified by terminology such as believe, may, intend, plan, will, expect, estimate, strategy, and similar expressions. These forward-looking statements are subject to material risks and uncertainties that are beyond our control, and many of these risks are discussed herein under the section entitled Risk Factors. Although our management believes that the assumptions underlying these forward-looking statements are reasonable, they do not assure or guarantee our future performance. Our actual results could differ materially from those contemplated by these forward-looking statements. The assumptions used for these forward-looking statements require considerable exercise of judgment, since they represent estimates of future events and are thus subject to material uncertainties involving possible changes in economic, governmental, industry, marketplace, and other circumstances. To the extent any assumed events do not occur, the outcome may vary materially from our anticipated or projected results. In light of these risks and uncertainties, there can be no assurance that the results and events contemplated by these forward-looking statements will in fact transpire. You are cautioned to not place undue reliance on these forward-looking statements, which speak only as of their dates. We do not undertake any obligation to update or revise any forward-looking statements. 3 Table of Contents PART I ITEM 1 BUSINESS Recent Status of the Company In order to fund and conduct our business over the past few years, we relied significantly on working capital obtained from private sales of our equity and convertible debt securities to various accredited investors. Due primarily to our continued substantial operating losses for the past several years, we recently have been unable to continue raising such working capital as needed to support adequately our business plan for future growth. And unless we are able to raise needed substantial additional funding to achieve significant future revenue growth, our current business model most likely will not succeed. The board is exploring raising more working capital through the issuance of common and preferred stock. We have engaged an investment banking firm that specializes in health care to advise on the appropriate structure and to assist raising capital. Effective September 18, 2022, the Board of Directors of registrant FISION Corporation, a Delaware corporation FISION replaced William Gerhauser as the Chief Executive Officer of FISION, and concurrent and immediately effective thereto, the Board of Directors appointed John Bode as Interim Chief Executive Officer to serve until his successor is appointed or elected. Mr. Bode is currently a director of FISION and will continue to serve on its Board of Directors. Michael Brown, a director of FISION, was also appointed as Chairman of the Board of Directors of FISION effective immediately. The Board of Directors appointed William Gerhauser chief executive officer on November 19, 2021. Previous to this appointment, our management consisted of our four directors Michael Brown, William Gerhauser, John Bode (independent), and Gregory Nagel (independent). Michael Brown served as our principal executive. Joshua Carmona was appointed Director and Chief Operating Officer upon the completion of the acquisition of Scoreinc.com on May 30, 2021. Mr. Gerhauser is the 100 owner of Capital Markets Solutions, LLC CMS through which the Company was managed by two executives under the terms of a Consulting Agreement between us and CMS. When this CMS Consulting Agreement expired in March 2020, the two CMS executives managing the Company resigned all management positions with the Company. Over the past two years, we have increased fixed cost cutting measures through outsourcing administrative, marketing and development functions to help manage working capital. At this time, we also no longer lease any office, administrative or operational facilities other than a virtual office location in Minneapolis on a monthly basis. Acquisition to Engage in Medical Ambulatory Surgery Center ASC Business In November 2020, the Company entered into an agreement to acquire 100 of the equity membership interests of Ft Myers ASC LLC Ft Myers ASC and ASC SoftDev LLC SoftDev from Capital Market Solutions, LLC CMS ), in exchange for the reimbursement of up to 200,000 of related out of pocket expenses. These two Florida limited liability companies were organized by CMS in the fall of 2020 to engage in the development and operation of a medical Ambulatory Surgery Center and are now wholly owned subsidiaries of the Company. Our agreement with CMS for this acquisition was set forth in full as Exhibit 10.1 to our Current Report on Form 8-K filed with the SEC on November 19, 2020. CMS is an affiliate of the Company and its largest shareholder, and is controlled by William Gerhauser, a director of the Company. This acquisition and its terms were specifically considered and approved by our two independent and disinterested directors, who also were advised by independent outside legal counsel. Neither Mr. Gerhauser nor any other representative of CMS participated in the vote of our Board of Directors to approve this acquisition. (See Note 11). 4 Table of Contents Fort Myers ASC was formed for the purpose of owning and operating in Ft Myers, Florida an Ambulatory Surgery Center, a medical facility specializing in elective same-day or outpatient surgical procedures, not including emergency surgery. SoftDev was formed for the purpose of developing software applications to support the medical procedures and operations of Ambulatory Surgery Centers, including Ft Myers ASC and others. SoftDev has engaged experienced software development consultants and others to assist in the development of its proprietary software platform and other business operations, including Rubicon Software Ltd. We expect to have completed our proprietary Ambulatory Surgery Center software platform applications prior to opening our Ft Myers Ambulatory Surgery Center. Acquisition of Score On April 1, 2021, the Company entered into a Memorandum of Understanding MOU with Scoreinc.com, Inc., a Puerto Rico Corporation Score and Joshua Carmona Carmona ), an individual who owned 100 of Score. This MOU contained the material terms of the acquisition by the Company of 100 of Score including its subsidiaries. On May 30, 2021, the Company entered into a definitive Purchase and Sale Agreement PSA with Score, Carmona, and VIP Solutions, LLC VIP ), a subsidiary of Score, and pursuant to the PSA, the Company acquired 100 of Score to become a wholly owned subsidiary of Fision Corporation. The Company also acquired certain assets of VIP listed in the PSA. Score is an Act 73 company under Puerto Rico law that is in the enterprise software space and currently provides business to business solutions for approximately 100 US companies in the credit repair sector. Mr. Carmona owned 100 of Score capital stock free and clear of all liens and encumbrances of any kind, and VIP owned its assets acquired by the Company free and clear of all liens and encumbrances of any kind. For accounting and general purposes, the date of acquisition of Score was considered to be the May 30, 2021 date of the PSA, although the final closing occurred in August 2021 only after a required certified audit of Score s business operations and financial position was completed and accepted by the Company. In consideration for this acquisition of Score, we issued to Carmona a Senior Secured Promissory Note for 500,000 substantially in the form attached as Exhibit 1 of the PSA, convertible into not more than ten (10) million shares of common stock of the Company at the higher of USD 0.05 per share or at the volume weighted average price (VWAP) over the last 10 trading days prior to conversion. The Company will also issue to Carmona not later than March 31, 2023 a second, unsecured promissory note in a form satisfactory to the Company and Carmona in an amount equal to Score s average gross revenue during calendar years ending 2021 and 2022, which will be convertible into not more than 10 million shares of common stock of the Company at USD 0.20 per share and will contain the usual and customary protections and adjustments for future corporate actions, including but not limited to pricing adjustments for reverse stock splits. We also appointed Carmona as a member of our Board of Directors and as our Chief Operating Officer. His sole compensation for these management services will be 50,000 per year paid at 12,500 quarterly in shares of restricted common stock of Fision as determined by the closing stock price on the last trading day of each calendar quarter. Background. FISION Corporation (the Company was incorporated in Delaware in 2010 under a former name, and conducted no active business operations until December 2015 when the Company merged with Fision Holdings, Inc., Minnesota Fision an operating Minnesota corporation based in Minneapolis. As a result of this 2015 Merger, Fision Holdings, Inc. became our wholly-owned subsidiary, and control of the Company was acquired by the pre-merger shareholders of Fision Holdings, Inc. In connection with this 2015 Merger, we issued an aggregate of 28,845,090 shares of our common stock to the former shareholders of Minnesota Fision, and also issued derivative securities to holders of Minnesota Fision outstanding options and warrants to purchase an aggregate of 3,868,575 additional shares of our common stock. As a result of this 2015 Merger, our pre-merger shareholders plus holders of our pre-merger derivative securities held less than five percent (5 of our total combined post-merger outstanding common stock plus reserved common stock for all derivative securities. The 2015 Merger was accounted for as a reverse merger and recapitalization. Accordingly, for financial reporting purposes, our Minnesota Fision subsidiary was the acquirer, and the Delaware parent was the acquired company. When used in this report, the terms the Company, Fision, we, us, and our, refer to FISION Corporation, a Delaware corporation and our wholly-owned operating subsidiary Fision Holdings, Inc., a Minnesota corporation. 5 Table of Contents Termination of Continuity Logic Merger and Subsequent Settlement - In late 2018 the Company entered into a Merger Agreement with Continuity Logic, L.L.C., a New Jersey limited liability company (the Continuity Logic Merger to effect a merger which would have resulted in our shareholders as a group and the equity holders of Continuity Logic as a group each owning 50 of the post-merger combined companies. A key provision of this Continuity Logic Merger provided that the parties and their representatives were required to raise enough working capital to support the post-merger funding requirements of the combined companies, and also that the merger be completed by December 31, 2018. Accordingly, in February 2019 Continuity Logic terminated this merger since it was not consummated by December 31, 2018. The Company no longer has any relationship or involvement with the business or future prospects of Continuity Logic. While the Continuity Logic Merger was pending, the Company made various bridge loans to Continuity Logic for working capital, of which a total balance of 905,500 has been long overdue and is still outstanding. These loans matured on August 31, 2019, bear interest at 6 per annum, and a substantial portion of the Notes is secured by a first-priority perfected security interest on the accounts receivable of Continuity Logic. The termination of the Continuity Logic Merger did not change or affect the continuing obligation of Continuity Logic to satisfy the payment of these loans to the Company. The Company took a reserve as of December 31, 2019 of the entire 905,500 plus accrued interest against the balance of these notes receivables. Under a court approved settlement agreement dated June 28, 2021, which Continuity Logic LLC has satisfied its obligations within the time periods specified in the settlement agreement and made payments totaling 567,993 for the year ended December 31, 2021, all of which was received in the third quarter. The Company recorded a gain on this settlement of 567,993 for the year ended December 31, 2021. Continuity Logic LLC was also required to pay the balance of the Company s unsecured claim plus additional accrued interested by April 30, 2022. Continuity Logic LLC satisfied all the required obligations of the settlement agreement by making payments totaling 431,040 in the second quarter of 2022. Subsequently, both companies entered into a mutual release. Business of Company We are an Internet platform technology company providing cloud-based software solutions to automate the marketing functions and activities of our customers and to provide credit repair tools to credit repair businesses. Our business is conducted through: i. our Minnesota Fision subsidiary based in Minneapolis, which since 2011 has created and offered software solutions to support marketing and sales enablement activities of both private businesses and public companies; ii. our Scoreinc.com subsidiary based in Puerto Rico, which was acquired on May 30, 2021 and provides software solutions including credit repair tools, strategies and services to credit repair businesses. Under ASC SoftDev LLC, we are using certain attributes of the two existing software platforms of the Company to assist in building and creating a new software platform to assist the efficiencies of the ASC ambulatory surgery center operations. Under Fort Myers ASC LLC, we are in the final stages of approval for the development and renovation for a four operating room orthopedic surgery center under the operating name Total Joint Orthopedic Surgery Center in Fort Myers, Florida. 6 Table of Contents Our Customers Our potential customer base is global and virtually unlimited, since our software solutions are totally cloud-based and readily scalable, and include a multitude of digital tools and solutions which can provide significant benefits to our customers on both platforms. We have received substantial recurring revenues from our primary customers for many years, and we regard our high percentage of recurring revenues to be particularly significant to our marketing strategy which emphasizes long-term relationships with our customers. Cloud-Based Platform Storage and operation of our software solutions platform along with the digital marketing and sales assets and related data of our customers are outsourced by us to reside and take place in the digital cloud. Providers of cloud services are typically referred to as virtual servers since they provide all digital data storage and software application services to their clients. Our cloud service provider is Microsoft s Azure Cloud, which leading cloud-based platform offers readily scalable, high quality and secure cloud services capable of satisfying any increasing demand or changing circumstances in the needs of our customers or us. We regard the hosting of our software applications, the ready digital cloud interface with our customers, and the storage of unlimited customer data provided by our premier cloud provider as being crucial to our operating strategy. Our major savings in expensive computer equipment, high salaried technology personnel, and costly security measures through our use of Microsoft s cloud is vital to our cost of doing business. Moreover, we believe that our experienced and leading cloud provider is more effective in delivering our software solutions to our customers than we could perform in any event. Research and Development The Company has committed substantial financial, personnel and other resources toward research and development efforts and activities related to the integration, commercialization and improvement of the Company s proprietary software platforms. We currently leverage both an in-house development team and outsourced consultants to assist in achieving our research and development objectives. Our Industry We have marketed and licensed our software products and services in the agile marketing segment of the broad software-as-a-service (SaaS) industry, with virtually all our revenues derived from our proprietary cloud-based marketing software platforms;. Employees We currently employ 11 employees through our Scoreinc.com subsidiary. We currently intend to hire additional human resources in Florida to assist in the administration and execution of our strategy related to the ASC development. Outsourcing We currently outsource our software platform maintenance and operations and our accounting and administrative functions to various experienced independent contractors. We believe that the software and other services provided by our outsourced contractors are adequate to service our current customers as required and to maintain our corporate functions in a professional manner. Marketing Model During our early years while our Fision software platform was being designed and developed, our marketing and sales efforts were directed toward local or regional small-to-medium sized companies having their operations conducted from one local or a couple regional facilities. Since our Fision platform was designed to be particularly beneficial for and adaptable to large size enterprises, however, in late 2017 we revised our marketing strategy and activities to target and sell our Fision marketing software primarily to large national or global corporate enterprises having many and widespread branches and operations. Although our marketing focus for the past few years toward large commercial enterprises has been effective in achieving a material base of such significant large customers, we have been unable to generate enough increased recurring revenues from them to support our related increased marketing, operational and administrative costs. Unfortunately, the increased length of time of the sales cycle necessary to close licensing contracts with large enterprises has been substantially longer and much more expensive than we anticipated or had incurred while marketing to smaller companies, resulting in a substantial decline in our revenues over the past few years. And since we failed to close and secure enough large enterprise customers to achieve the necessary increased revenues to support any further material marketing, we terminated all sales and marketing personnel during 2019. We have been attempting to raise material working capital to support a marketing campaign to attract additional large enterprise customers, but there is no assurance any such future funds will be available to us. 7 Table of Contents ITEM 1A RISK FACTORS An investment in our common stock is speculative and involves a high degree of risk. You should carefully consider the following risks, along with all of the other information contained in this Annual Report, before considering making an investment decision to purchase any of our securities. If one or more of the following risks occur, our business, financial condition and results of operations could suffer materially. In that case, the price of our common shares in any trading market could decline materially, and you could lose a substantial part or all of your investment. Risks Related to Our Business We have a limited operating history and substantial accumulated losses, and we anticipate incurring continued losses. Our business must be considered and evaluated in light of the risks, expenses, and problems frequently encountered by companies in their early stages of development and commercialization of products, and particularly companies like us participating in new and rapidly evolving markets. These risks include our failure to anticipate and adapt to new technology or changing market conditions, our inability to secure enough future material customers to achieve profitability, the rejection of or lack of satisfaction with our services and products by our existing or potential customers, our inability to obtain additional capital when needed from time to time, our inability to protect our intellectual property, and any development of equal or superior products or services by our competitors. Accordingly, there can be no assurance we will be successful in addressing these risks. We incurred operating losses of (1,230,029) for the year ended December 31, 2021 and net losses of (2,491,533) for the year ended December 31, 2020. Since our inception in 2010, we have an accumulated deficit of (31,474,016) as of December 31, 2021. Moreover, we expect to continue operating at a loss during 2022 and likely even beyond 2022. There is no assurance based on our past business experience to support any belief that we can become profitable or sustain profitability in the future. There can be no assurance that we can generate significant revenue growth, or that any revenue growth that we achieve can be sustained. To any extent that increases in our operating expenses precede or are not soon followed by increased revenues, our business, results of operations and financial condition would be adversely affected materially. If we are unable to obtain significant future financing from time to time, our development and operations will encounter serious delays or could even result in the complete failure of our business. Our ability to become commercially successful and profitable will depend largely on our being able to continue raising significant additional financing from time to time in the future. If we are unable to raise additional financing through equity and/or debt sources as needed, we would not be able to succeed in our commercial operations, which eventually could result in our failure. There is no assurance any such additional funds will be available on terms satisfactory to us, if at all. We have a significant amount of debt, much of which has matured, and this could limit or even eliminate recovery of your investment if we fail to reach substantial profitability or otherwise resolve or satisfy our outstanding debt. We have a substantial amount of indebtedness in the form of various Notes Payable. As of December 31, 2021, we had approximately 2.5 Million of outstanding Notes Payable, much of which are now due, or even due on demand. There is no assurance that we will be able to convert our outstanding debt into common stock, or to satisfy this debt through new financing or refinancing, or to achieve profitability sufficient to make future payments to satisfy this debt. If we cannot satisfy or resolve our substantial outstanding indebtedness, such failure would have a significant adverse effect on our business and financial condition and could even cause a total failure of our business. 8 Table of Contents We may not be able to continue as a going concern. Our operating expenses have been funded by various sources including sales of our equity and debt securities, limited revenues from customer contracts, deferral of management salaries, and stock-based payment to management and certain service providers and consultants. Due to this situation and our ongoing substantial losses, there is substantial doubt whether our business can continue as a going concern to support its commercial operations and satisfy its outstanding debt and other obligations. Our management is in the process of attempting to obtain additional significant financing to enable us to continue as a going concern, but there is no assurance that adequate funds will become available to allow us to continue as a going concern. Our ability to generate future material revenues will depend upon a number of factors, some of which are beyond our control. These factors include the rate of acceptance of our software products, competitive pressures in our industry, adapting to changes in technology in our industry, and general economic trends. We cannot forecast accurately what our revenues will be in future periods. Our operating results have fluctuated and been difficult to predict, and our results could continue to fluctuate and be unpredictable in the future. Our operating results have been difficult to predict, have historically fluctuated, and may vary significantly from quarter to quarter due to a variety of factors, many of which are beyond our control. Accordingly, comparing our operating results on a period-to-period basis may not be meaningful. Factors that may affect our quarterly operating results may include: any material changes in demand for our software solutions; the introduction of new technologies by competitors; the nature of our variable and unpredictable sales cycle; changes in the number, availability and quality of competing products; the timing and amount of sales and marketing expenses incurred by us to attract new customers; changes in the economic or business prospects of our customers or the economy generally; changes in our pricing policies or the pricing policies of our competitors; changes in governmental regulation of the Internet, wireless networks and mobile platforms; unforeseen costs necessary to improve and maintain our technologies; and costs related to any acquisitions undertaken by us. We face significant challenges in obtaining market acceptance of our products and establishing our Fision brand. Our success depends primarily upon the acceptance of our software platform and our brands by new customers, most of who are not familiar with and have not used our products, and also do not recognize our brand and corporate identity. Acceptance of our software solutions by new customers will depend on many factors including price, reliability, performance, service accessibility and effectiveness, and our ability to overcome existing loyalties to established brands. There is no assurance we will be able to meet these challenges successfully. We depend on a small number of customers for a substantial portion of our revenues, and any material reduction in the use of our software platform by our major customers could reduce our revenues significantly. For the fiscal years ended December 31, 2021 and 2020, two customers and three customers, respectively, each accounted for more than 10 of our revenues. A significant reduction for any reason in the use of our software solutions by one or more of our major customers could harm our business materially. We currently have no fulltime management or key employees, which harms our business substantially. Our future success will depend significantly upon our financial ability to attract and sustain fulltime employment of competent management and other key employees. Our future business will be dependent on hiring additional qualified key personnel, and if they are not available when needed or we cannot obtain sufficient working capital to afford them if available, our future growth and prospects would suffer materially. 9 Table of Contents We operate in a highly competitive industry, and we may not be able to compete successfully. Our market is highly competitive, with many companies providing solutions competitive to our software platform. Well-known and established competitors include Marketo (Adobe), Eloqua (Oracle), Unica (IBM), Hubspot, ExactTarget (Salesforce), Aprimo, Responsys and Silverpop. We expect additional material competitors to emerge in the future from time to time. Most of our current and potential competitors have significantly more personnel, financial, technical, marketing and other resources than we possess, and accordingly they are able to devote substantially greater resources than us to development, marketing, sales, and support of their products and services. We lack many things which most of our competitors have, including an established brand and name recognition, existing relationships with a large customer base, the ability to undertake costly and extensive marketing campaigns, and large customer support teams. Software solutions and demands in our industry often change significantly, and our future success will depend materially on our ability to anticipate and respond to such changes. If we cannot adjust, develop and compete effectively with changing software solutions and products as and when they are introduced in our industry, our ability to generate meaningful revenues or achieve profitable operations would be impaired substantially. Our current and potential competitors may develop and offer new software technologies that render our products less competitive, unmarketable or even obsolete. In addition, if any competitors develop products with similar or better functionality than our solutions, we may need to decrease the prices for our products to remain competitive, which could result in a material reduction in our margins and a corresponding material negative effect on our operating results and financial condition. We may not be able to address and solve satisfactorily the many competitive pressures and challenges facing us in our industry, and our failure to do so would harm substantially our business, operating results and financial condition. Our market may develop more slowly than we expect, which could harm our business. Development of marketing software solutions is an emerging market, and our current and future customers may ultimately find our software platform to be less effective than anticipated for promoting their products or services, which as a result could cause them to reduce their spending on our software solutions. Failure to manage growth effectively could harm our business seriously. Any failure by our management or other key personnel to manage any future growth properly could impair our ability to deliver our software solutions in a timely fashion, to fulfill existing customer commitments, or to attract and retain new customers. Interruption or failure of information technology and communications systems used by us could hurt our ability to provide our services effectively, which could damage our reputation and harm our operating results. The availability to our customers of our products and services depends on the continuing and efficient operation of information technology and communications systems and infrastructure used by us, and most particularly on the cloud-based service provider facility which hosts our Fision software platform. These electronic systems are vulnerable to damage or interruption from earthquakes, vandalism, sabotage, terrorist attacks, floods, fires, power outages, telecommunications failures, and computer viruses or other deliberate attempts to harm the systems. The occurrence of a natural or intentional disaster, any decision to close a facility we are using without adequate notice, or particularly an unanticipated problem at our cloud-based virtual server facility, could result in harmful interruptions in our service, resulting in damage to our customers and brand. 10 Table of Contents We may not be able to adequately protect or enforce our intellectual property (IP) rights. Our success and competitive position will depend materially on our ability to protect our intellectual property (IP), including trademarks, trade names, patent rights and trade secrets. Despite our efforts to protect these proprietary rights, unauthorized parties may attempt to copy aspects of our intellectual property or to wrongly obtain and use our information technology. In addition, competitors may independently develop substantially equivalent intellectual property. If we do not effectively protect our intellectual property, or they become marginalized or valueless due to developments by a competitor, our business could suffer materially. Moreover, litigation may be necessary to enforce our intellectual property rights, to protect our trade secrets, or to determine the validity and scope of the proprietary rights of others. Any such litigation could result in substantial costs and diversion of our resources, which could cause serious harm to our business, operating results and financial condition. We have obtained certain patents from the United States Trademark and Patent Office (USTPO) covering certain technological aspects of our Fision software platform. We also have filed certain other patent claims with the USTPO and we expect to receive patent protection regarding these other pending patent claims. Although we regard the proprietary software technology covered by our granted and pending patents to be significant for our future business, there is no assurance that any granted or pending patents will provide us with any significant proprietary protection. We could incur substantial costs and disruption to our business as a result of any infringement claim brought against us involving intellectual property of another party, which could harm our business and operating results. We cannot predict whether any third-party claims will occur alleging that we have infringed their intellectual property rights, and if any such claims occur whether they will substantially harm our business and operating results. If we are forced to defend any infringement claims, whether with or without merit or even if determined in our favor, we may face costly litigation and diversion of our management and other personnel. Moreover, any outcome of a claim resulting in our having infringed another s intellectual property rights may require us to pay significant damages and attorney fees; to cease licensing our Fision software platform; to expend substantial development resources to redesign our products; or to enter into potentially unfavorable royalty or license agreements to use third-party technologies, which may not be available on terms acceptable to us, if at all. The time and resources required from us to resolve any such disputes or litigation, even if resolved in our favor, could harm our business, operating results, financial condition, brand and reputation. Risks Related to Our Common Stock Our common stock has been downgraded to the OTC Expert Market and a s a practical matter, the Company s stockholders no longer have a public trading market for their shares of common stock. OTC Markets Group has downgraded trading in the Company s common stock to the OTC Expert Market. See Item 5 of this Form 10-K.The Expert Market is available only for unsolicited quotes, meaning broker-dealers may use the Expert Market to publish unsolicited quotes representing limit orders from retail and institutional investors who are not affiliates or insiders of the issuer. Quotations in Expert Market securities are only made available to broker-dealers, institutions and other sophisticated investors.Because of the restrictions imposed on securities quoted on the Expert Market, most investors will not be able to publicly sell their shares. Additionally, they will not have access to bid and ask prices or other information, including trading volume. As such, Expert Market shares are illiquid and, as a practical matter, the Company s stockholders no longer have a public trading market for their shares of common stock. The Company is in the process of submitting the missed that resulted in the downgrade of the Company s common stock to the Expert Market. Upon submission of the missed filings, the Company currently intends to develop a plan, as soon as economically feasible, to return the Company s common stock to the OTC Pink market or other appropriate listing service or exchange. The Company s common stock is expected to continue to be traded on the OTC Expert Market, and the Company is currently considering alternatives that will provide additional trading opportunities in the Company s common stock at the earliest opportunity, including through submission of a Form 211 and application to upgrade to the Pink Current Tier of OTC Markets Group. However, there can be no assurance that such an application will be filed by the Company or approved by OTC Markets Group and the Company s common stock accepted for listing by the OTC Markets Group, or that a market for the Company s shares will develop. The current public trading market for our common stock has fluctuated widely. There is no assurance that the trading market for our common stock will become more stable, or will continue to be sustained. Moreover, there remains a significant risk that our common stock price may fluctuate substantially in the future in response to various factors including the following: Variations in our operating results. Departures or additions of management or other key personnel. Announcements of significant acquisitions or strategic joint ventures. Announcements of significant capital commitments or transactions. Announcements of significant patent or other technological matters. Substantial sales of our common stock in the open market. Any significant litigation matters. Announcements of new product developments. Gain or loss of significant customers. Natural disasters, acts of war, terrorism, or pandemics including the ongoing Coronavirus (COVID-19). General market conditions or specific political and economic conditions. 11 Table of Contents Our common stock is subject to the penny stock rules of the Securities and Exchange Commission, which may make it more difficult for our stockholders to sell their common stock. Under SEC rules, a penny stock includes any equity security having a market price of less than 5.00 per share, subject to certain limited exceptions. Regarding any transaction involving a penny stock, these rules require a broker or dealer to approve a person s account for transactions in penny stocks, and also to receive from the person a written consent to the transaction and its terms. The broker or dealer also must obtain financial information and investment experience objectives of the person and make a reasonable determination that the transactions in penny stocks are suitable for that person and that the person is competent to evaluate the risks of penny stocks. Prior to the transaction, the broker or dealer also must deliver a disclosure schedule containing the rights and remedies available to the person, the basis on which the broker or dealer made the suitability determination, and the receipt of the related signed agreement from the person. Due to these penny-stock rules, brokers and dealers may be less willing to execute transactions in penny stock securities, and this may make it more difficult for shareholders to dispose of our common stock, which could affect adversely the market value of our common stock. Because we became a publicly-held company by means of a reverse merger with an inactive public company, it may be difficult for us to attract the attention of brokerage firms and financial analysts. Due to our becoming public through a reverse merger, we currently have very limited relationships or public stock trading history with any brokerage firm, underwriter or financial analyst. Accordingly, there may be little or no incentive for securities analysts of brokerage and other financial firms to provide investment coverage of us or to recommend the purchase of our common stock. Being a public company results in additional expenses and diverts management s attentions. Our business must bear the expenses associated with being a public company including being subject to the reporting requirements of the Securities Exchange Act of 1934. These requirements generate significant accounting, legal and financial compliance costs, and may place significant strain on our personnel and resources. As a result, management s attention may be diverted from other significant business concerns, which could have an adverse material effect on our business, financial condition and results of operations. Applicable and extensive regulatory requirements, including those of the Sarbanes-Oxley Act of 2002, may make it difficult for the Company to retain or even attract qualified officers and directors, which could adversely affect our business and our future relationships in the investment community. Because of the many and ever increasing rules and regulations governing publicly-held companies, it may be difficult for us to attract and retain those qualified officers and directors necessary for effective management. The Sarbanes-Oxley Act particularly added demanding new rules and regulations and the expansion and strengthening of existing rules and regulations, including required certifications by our principal executive officers. The SEC, FINRA and the stock exchanges also continue to add to or expand their rules frequently. This extensive and ongoing regulatory situation regarding the adoption of new rules and regulations by these various agencies has created a real, or at least perceived, increased personal risk to members of management of public companies, which may deter qualified candidates from accepting roles as directors or officers of our Company. If we are unable to attract and retain future qualified officers and directors as needed, the management of our business and our future ability to obtain a listing of our securities on a national stock exchange could be adversely affected materially. 12 Table of Contents If we fail to establish and maintain an effective system of internal controls, we may not be able to report our financial results accurately or detect fraud. In that event, investors and the financial community could lose confidence in our financial reporting, which in turn may affect adversely the trading price of our stock, or otherwise harm our financial condition. We must maintain effective internal controls to provide reliable financial reports and detect any fraud or material weaknesses in our internal controls. Any failure by us to implement the changes necessary to maintain an effective system of internal controls could harm our operating results materially and also cause investors and financial analysts to lose confidence in our reported financial information. Even if we resolve effectively any such material weaknesses in our internal controls, this still may not prevent all potential errors, since any control system, regardless of its design, can provide only reasonable and not absolute assurance that the objectives of the control system will be achieved. Ownership and related voting power of our shareholders is concentrated in our affiliates and management. At December 31, 2021, our affiliates and directors own approximately 20 of our common stock (or 28 if they exercised warrants held by them), which concentrated ownership by them could adversely affect the price of our common stock. Any material common stock sales by our insiders or affiliates, for example, could result in a decline in the price of our common stock. This substantial concentration of stock ownership also most likely affords our affiliates and management the ability to control all matters requiring stockholder approval including the election of all directors, and the approval of mergers, acquisitions or other significant corporate transactions. Any person acquiring our common stock most likely will have no effective voice in the management of our company. This ownership concentration also could delay or prevent a change of control of the Company, which could deprive our stockholders from receiving a premium for their common shares. We do not intend to pay dividends for the foreseeable future. We have not paid any dividends on our common stock to date, and we do not anticipate paying any such dividends in the foreseeable future. We anticipate that any earnings experienced by us will be retained to finance the implementation of our operational business plan. Our Articles of Incorporation allow for our Board of Directors to create and designate new series of our preferred stock without any approval of our shareholders, which could diminish the rights of holders of our common stock. Our Board of Directors has the authority to fix and determine the relative rights and preferences of our authorized preferred stock without further common stockholder approval for issuance. Accordingly, our directors could authorize preferred shares, for example, that would grant a preference over common shareholders to our assets upon liquidation, or grant voting power and rights superior to those of common shares, or grant rights to preferred stock to accumulate and receive dividend payments before any dividend or other distribution to common shareholders, or grant special redemption terms and rights prior to any redemption of common shares, or grant rights convertible at favorable terms into common stock. Granting one or more of these or other preference rights to preferred stock could adversely affect the rights of our common stockholders including decreasing the relative voting power of our common stock and causing substantial dilution to our common stockholders. We have no outstanding preferred stock and no present intention to designate or issue any series of our preferred stock. 13 Table of Contents The elimination of monetary liability against our directors and executive officers under Delaware law and the existence of indemnification rights held by them granted by our bylaws may result in substantial expenditures by us and may discourage lawsuits against our directors and officers. Our articles of incorporation eliminate the personal liability of our directors and officers to the Company and its stockholders for damages for breach of fiduciary duty to the maximum extent permissible under Delaware law. These provisions and resultant costs may discourage us, or our stockholders through derivative litigation, from bringing a lawsuit against any of our current or former directors or officers for any breaches of their fiduciary duties, even if such legal actions, if successful, might benefit us or our stockholders. In addition, our bylaws provide that we are obligated to indemnify our directors or officers to the fullest extent authorized by Delaware law for costs or damages incurred by them involving legal proceedings brought against them relating to their positions with the Company. These indemnification obligations could result in our incurring substantial expenditures to cover the cost of settlement or damage awards against our directors or officers. Delaware law contains anti-takeover provisions which could deter or even prevent acquisition attempts of our company that may be beneficial to our stockholders. Provisions of Delaware law could make it more difficult for a third party to acquire the Company, even if the acquisition would be beneficial to our stockholders. For example, Section 203 of the Delaware General Corporation Law may make the acquisition of our company and the removal of our incumbent officers and directors more difficult by prohibiting stockholders holding 15 or more of our outstanding common stock from acquiring us, without the consent of our board of directors, for at least three years from the date they first owned at least 15 of our common stock. Natural disasters and other events beyond our control could materially adversely affect us. Natural disasters or other catastrophic events may cause damage or disruption to our operations, international commerce and the global economy, and thus could have a strong negative effect on us. Our business operations are subject to interruption by natural disasters, fire, power shortages, pandemics (including the ongoing Coronavirus (COVID-19) epidemic) and other events beyond our control. Although we maintain certain limited disaster response plans, such events could make it difficult or impossible for us to deliver our services to our customers, and could decrease demand for our services. ITEM 1B UNRESOLVED STAFF COMMENTS Not applicable. 14 Table of Contents ITEM 2 PROPERTIES Our principal telephone and other administrative office functions are currently being outsourced to an independent contractor in Minneapolis, Minnesota providing such virtual office services to various business entities, and located at 1650 West End Boulevard, Suite 100, Minneapolis, MN 55416. For these administrative office services, we pay a monthly fee of 322 which includes a storage fee for certain computer servers and other technology equipment and supplies owned by us. We do not own any real estate. We believe that such current outsourced office facilities and services are adequate to satisfy our current office and administrative operations. ITEM 3 LEGAL PROCEEDINGS In late 2018 the Company entered into a Merger Agreement with Continuity Logic, L.L.C., a New Jersey limited liability company (the Continuity Logic Merger to effect a merger which would have resulted in our shareholders as a group and the equity holders of Continuity Logic as a group each owning 50 of the post-merger combined companies. A key provision of this Continuity Logic Merger provided that the parties and their representatives were required to raise enough working capital to support the post-merger funding requirements of the combined companies, and also that the merger be completed by December 31, 2018. Accordingly, in February 2019 Continuity Logic terminated this merger since it was not consummated by December 31, 2018. The Company no longer has any relationship or involvement with the business or future prospects of Continuity Logic. While the Continuity Logic Merger was pending, the Company made various bridge loans to Continuity Logic for working capital, of which a total balance of 905,500 has been long overdue and is still outstanding. These loans matured on August 31, 2019, bear interest at 6 per annum, and a substantial portion of the Notes is secured by a first-priority perfected security interest on the accounts receivable of Continuity Logic. The termination of the Continuity Logic Merger did not change or affect the continuing obligation of Continuity Logic to satisfy the payment of these loans to the Company. The Company took a reserve as of December 31, 2019 of the entire 905,500 plus accrued interest against the balance of these notes receivables. Under a court approved settlement agreement dated June 28, 2021, which Continuity Logic LLC has satisfied its obligations within the time periods specified in the settlement agreement and made payments totaling 567,993 for the year ended December 31, 2021, all of which was received in the third quarter. The Company recorded a gain on this settlement of 567,993 for the year ended December 31, 2021. Continuity Logic LLC was also required to pay the balance of the Company s unsecured claim plus additional accrued interested by April 30, 2022. Continuity Logic LLC satisfied all the required obligations of the settlement agreement by making payments totaling 431,040 in the second quarter of 2022. Subsequently, both companies entered into a mutual release. During the period, we settled and satisfied the complaint against us in a Minnesota District Court regarding our debt to Decathlon Alpha L.P. We used 60,000 of these Continuity Logic LLC settlement proceeds to make an initial payment on a this settled complaint with Decathlon Alpha L.P., with a final payment of 117,500 paid on September 15, 2022. ITEM 4 MINE SAFETY DISCLOSURES Not applicable. 15 Table of Contents PART II ITEM 5 MARKET FOR REGISTRANT S COMMON EQUITY, RELATED STOCKHOLDER MATTERS AND ISSUER PURCHASES OF EQUITY SECURITIES Market Information Our common stock has historically been publicly traded in the over-the-counter market, and has been quoted and traded for years on either the OTCQB Tier or the Pink Current Tier of OTC Markets Group under the symbol FSSN . Our common stock is thinly traded, and there is no assurance that our common stock will trade more actively in the future. On July 20, 2022, the Company was notified by OTC Markets Group that trading in the Company s common stock would be downgraded to the OTC Expert Market because the Company s disclosures were not current and publicly available under SEC Rule 15c2-11 as a result of the Company s delinquent Form 10-K for the year ended December 31, 2021 and Form 10-Q for the three months ended March 31, 2022. In essence, Rule 15c2-11 prohibits dealers from publishing quotations for OTC securities to quotation mediums without first reviewing certain issuer financial information and ensuring that information is current and publicly available before quoting that security. Accordingly, the Company s common stock was downgraded to the Expert Market effective with the open of business on July 21, 2022. Since that date, the Company s common stock has been eligible for trading on the OTC Expert Market. The Expert Market is available only for unsolicited quotes, meaning broker-dealers may use the Expert Market to publish unsolicited quotes representing limit orders from retail and institutional investors who are not affiliates or insiders of the issuer. Quotations in Expert Market securities are only made available to broker-dealers, institutions and other sophisticated investors. Because of the restrictions imposed on securities quoted on the Expert Market, most investors will not be able to publicly sell their shares. Additionally, they will not have access to bid and ask prices or other information, including trading volume. As such, Expert Market shares are illiquid and, as a practical matter, the Company s stockholders no longer have a public trading market for their shares of common stock. A company relegated to the Expert Market will be able to reapply for listing on the OTC Markets through the filing of a new Form 211 with FINRA once the Company has made current information available and is in compliance with Rule 15c2-11. Prior to the filing of this Form 10-K, the Company was not current with its SEC reporting obligations with respect to the Annual Report on Form 10-K for fiscal year 2021 (the Missed Annual Report ), and the Quarterly Reports on Form 10-Q for the periods ending March 31, 2022 and June 30, 2022 (the Missed Quarterly Reports, and together with the Missed Annual Report, the Missed Filings ). The Company is in the process of satisfying the required reporting requirements. This Form 10-K represents the Missed Annual Report, and the Company currently expects to file the Missed Quarterly Reports by January 31, 2023. The submission of those filings will complete the process of preparing the Missed Filings that resulted in the downgrade of the Company s common stock to the Expert Market. Upon submission of the Missed Filings, the Company currently intends to develop a plan, as soon as economically feasible, to return the Company s common stock to the OTC Pink market or other appropriate listing service or exchange. The Company s common stock is expected to continue to be traded on the OTC Expert Market, and the Company is currently considering alternatives that will provide additional trading opportunities in the Company s common stock at the earliest opportunity, including through submission of a Form 211 and application to upgrade to the Pink Current Tier of OTC Markets Group. However, there can be no assurance that such an application will be filed by the Company or approved by OTC Markets Group and the Company s common stock accepted for listing by the OTC Markets Group, or that a market for the Company s shares will develop. Since January 1, 2020, the high and low sale prices per share for our common stock as reported by OTC Markets Group are shown below, which quotations do not reflect retail mark-up, markdown or commission: High Low Price Price January March, 2020 0.0166 0.0021 April June, 2020 0.006 0.0021 July September, 2020 0.0186 0.0036 October December, 2020 0.033 0.005 January March, 2021 0.040 0.019 April June, 2021 0.088 0.023 July September, 2021 0.037 0.020 October December, 2021 0.033 0.017 January March, 2022 0.024 0.010 April June 2022 0.017 0.005 July September 2022 0.0111 0.0002 October December 22, 2022 0.001 0.0009 The last sale price of our common stock as reported by OTC Markets Group on December 22, 2022 was 0.001 per share. Stockholders As of December 22, 2022, there were 560 holders of record of our common stock. Dividend Policy We have never paid any cash dividends on our common stock. For the foreseeable future, we intend to retain any future earnings for the development and growth of our business. Any future payment of dividends will be determined by the Board of Directors on the basis of various factors, including our results of operations, financial condition, capital requirements, and investment or acquisition opportunities. Description of Securities Common Stock Our Board of Directors is authorized to issue up to 500,000,000 shares of common stock, par value .0001 per share, and as of December 22, 2022, there were 462,878,589 shares of our common stock outstanding. Holders of our common stock are entitled to one vote per share on all matters to be voted upon by stockholders. Holders of common stock are entitled to receive ratably such dividends, if any, as may be declared by our Board of Directors out of funds legally available for dividends. Upon the liquidation, dissolution or winding up of the Company, holders of common stock are entitled to share ratably in all our assets legally available for distribution after payment of all our debts and liabilities and liquidation preference of any outstanding preferred stock. Holders of common stock have no preemptive, subscription, redemption or conversion rights, and they have no rights for cumulative voting regarding any election of directors. 16 Table of Contents Preferred Stock The Company is also authorized to issue up to 20,000,000 shares of preferred stock, par value .0001 per share. Our Board of Directors has the authority, without stockholder approval, to issue these preferred shares in one or more designated series, to establish the number of shares in each series, and to fix the preferences, powers, dividends, and voting or conversion rights of any preferred shares, which could dilute the voting power or other rights of holders of our common stock. We have no outstanding preferred stock, and we have no current plans to designate terms of or issue any preferred stock. Transfer Agent Our transfer agent is Globex Transfer, LLC, 780 Deltona Blvd., Suite 202, Deltona, FL 32725, phone number (813) 344-4490. Our transfer agent is registered under the Securities Exchange Act of 1934. Rule 144 In general, under Rule 144 of the Securities Act of 1933, as amended, any person (or persons who are aggregated) including persons who are affiliates, whose restricted common stock has been fully paid for and held for at least six months from the later of the date acquired from us or from an affiliate of us, may sell such common stock in broker s transactions or directly to market makers, provided the number of shares sold in any three-month period may not exceed one percent of our then outstanding shares of common stock. Sales under Rule 144 are also subject to certain notice requirements and the availability of current public information about us. After one year has elapsed since the date of purchase or other acquisition of such common shares from us or an affiliate of us, persons who are not affiliates of us may sell their shares under Rule 144 without any limitation. Future sales of our common stock under Rule 144 or otherwise could negatively impact the price of our common stock in any public market where it is traded. We cannot estimate the number of shares that may be sold in the future by our existing shareholders or the effect, if any, that sales of shares by our stockholders will have on the market price of our common stock prevailing from time to time. Securities Authorized for Issuance Under Equity Compensation Plans The following table sets forth, as of December 31, 2021, our securities authorized for issuance under any equity compensation plans approved by our stockholders as well as any equity compensation plans not approved by our stockholders. Number of securities Number of remaining securities Weighted- available for to be issued average future upon exercise exercise issuance of price of under equity outstanding outstanding compensation options, options, plans warrants and warrants and (excluding rights rights column (a)) column (a) column (b) column I Equity compensation plans approved by -0- 4,600,000 security holders (1) shares -- shares Equity compensation plans not approved by 47,209,571 -0- security holders (2) shares .15/share shares _____________________ (1) All former options granted under our 2011 Plan have expired pursuant to their terms. Represents warrants granted on a one-time basis to affiliates, consultants and advisors under individual contracts or award arrangements. These grants vary in term, number of shares, and exercise price as determined by the Board of Directors to be in the best interests of the Company at the effective date of the grant of each award. (2) Represents warrants granted on a one-time basis to affiliates, consultants and advisors under individual contracts or award arrangements. These grants vary in term, number of shares, and exercise price as determined by the Board of Directors to be in the best interests of the Company at the effective date of the grant of each award. 17 Table of Contents Purchases of Equity Securities by the Issuer and Affiliated Purchasers. None. ITEM 6 SELECTED FINANCIAL DATA Not applicable ITEM 7 MANAGEMENT S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONS. The financial data referred to in the following discussion and analysis is derived from our audited financial statements for the fiscal years ended December 31, 2021 and 2020, which are included in this Annual Report. These financial statements have been prepared and presented in accordance with generally accepted accounting principles (GAAP) in the United States. The following discussion and analysis of our financial data is only a summary and you should read and consider it in conjunction with our financial statements and their related notes. This discussion contains forward-looking statements reflecting our current expectations that involve risks and uncertainties. Our actual results and the timing of events may differ materially from those contained in our forward-looking statements due to a number of factors, including those discussed in the section entitled Risk Factors and elsewhere in this Annual Report on Form 10-K. Overview We are an Internet platform technology company providing cloud-based software solutions to automate the marketing functions and activities of our customers and to provide credit repair tools to credit repair businesses. Our business is conducted through: iii. our Minnesota Fision subsidiary based in Minneapolis, which since 2011 has created and offered software solutions to support marketing and sales enablement activities of both private businesses and public companies; iv. our Scoreinc.com subsidiary based in Puerto Rico, which was acquired on May 30, 2021 and provides software solutions including credit repair tools, strategies and services to credit repair businesses. Under ASC SoftDev LLC, we are using certain attributes of the two existing software platforms of the Company to assist in building and creating a new software platform to assist the efficiencies of the ASC ambulatory surgery center operations. Under Fort Myers ASC LLC, we are in the final stages of approval for the development and renovation for a four operating room orthopedic surgery center under the operating name Total Joint Orthopedic Surgery Center in Fort Myers, Florida. Revenue Model Our revenue model is primarily based on prescribed software licensing fees received by us on a regular monthly basis from customers which are under written licensing agreements with us. Because of the long-term nature and the substantial expense commitment required by each new customer to enter into a binding licensing agreement with us, the sales cycle involved in our revenue model is quite lengthy. Accordingly, the unpredictable and different timing involved from customer to customer to procure our licensing contracts has prevented us from receiving consistent overall revenues or accurately forecasting our future revenue stream. 18 Table of Contents We generate our revenues primarily from recurring monthly payments from customers having a license from one to three years to access and use our proprietary marketing software platform, which payments include fees based on actual use of the Fision platform. We also receive from each new customer a prescribed one-time set-up and integration fee payable to us at the outset of the license. And we receive certain secondary fees from time to time for customized software development projects, and for processing emails for certain customers. Marketing Model We have marketed and licensed our proprietary software products primarily through direct sales by our management and other in-house personnel, and also secondarily through experienced and recognized independent sales agencies. We generate our revenues primarily from such software licensing contracts, and we currently have six (6) licensed customers using our Fision platform. We market and sell our products and services in the marketing software segment of the broader software-as-a-service (SaaS) industry. Intellectual Property (IP) In 2017, we were granted Patent No. US 9,639,551 B2 from the United States Patent and Trademark Office (USPTO), and in 2018 we were granted Patent No. US 9,984,094 B2 from the USPTO, and another granted Patent in 2019 Patent No. US 10,235,380 B2, from the USPTO which were titled Computerized Sharing of Digital Asset Localization Between Organizations. We also have an additional patent claim involving our software technology filed and pending with the USPTO. Inflation and Seasonality We do not consider our operations and business to be materially affected by either inflation or seasonality. Critical Accounting Policies and Estimates Principles of Consolidation Regarding our wholly-owned subsidiaries, our financial statements are presented on a consolidated basis with all intercompany transactions and balances eliminated in consolidation. Use of Estimates The preparation of consolidated financial statements in conformity with generally accepted accounting principles requires management to make estimates and assumptions that affect the reported amounts of assets and liabilities and disclosure of contingent assets and liabilities at the date of the financial statements and the reported amounts of revenues and expenses during the reporting period. Accordingly, actual results could differ materially from those estimates and assumptions. Such estimates include management s assessments of the carrying value of certain assets, useful lives of assets, derivative securities, fair value of financial instruments, and related depreciation and amortization methods applied. 19 Table of Contents Accounts Receivable Accounts receivable related to the products and services sold are recorded at the time revenue is recognized and are presented on the balance sheet net of allowance for doubtful accounts. The ultimate collection of the receivable may not be known for several months after services have been provided and billed. We have established an allowance for doubtful accounts based upon factors pertaining to the credit risk of specific customers, analyses of current and historical cash collections, and the aging of receivables. Delinquent accounts are written-off when the likelihood for collection is remote and/or when we believe collection efforts have been fully exhausted and we do not intend to devote any additional efforts in an attempt to collect the receivable. We adjust our allowance for doubtful accounts balance on a quarterly basis. Product Development and Support We expense all our product development and support operations and activities as they occur. During the fiscal year ended December 31, 2021 we incurred total expenses of 452,493 for such development and support. In comparison, during the fiscal year ended December 31, 2020 we incurred total expenses of 340,232 for product development and support. Property and Equipment Property and equipment are capitalized and stated at cost, and any additions, renewals or betterments are also capitalized. Expenditures for maintenance and repairs are charged to earnings as incurred. If property and equipment are retired or otherwise disposed of, the related cost and accumulated depreciation are removed from our accounts, and any gain or loss is included in operations. Depreciation of property and equipment is provided using the straight-line method with estimated lives as follows: Furniture and fixtures 5 years Computer and office equipment 5 years Derivative Securities We evaluate all of our agreements and financial instruments to determine if they contain features that qualify as embedded derivatives. For any derivative financial instruments accounted for as liabilities, they initially will be accounted for at fair value and if necessary re-valued at each reporting date, with any changes in fair value reported in our statements of operations. For any stock-based derivative financial instruments or securities, we use an option pricing model to value them at inception and on any subsequent valuation dates. The classification of derivative instruments, including whether they should be recorded as liabilities or as equity, is evaluated at the end of each reporting period. Derivative instrument liabilities are classified in our balance sheet as current or non-current based on whether or not net-cash settlement of the derivative instrument could be required within 12 months of the balance sheet date. Fair Value of Financial Instruments FASB ASC Topic 820 requires disclosure of and defines fair value of financial instruments, and also establishes a three-level valuation hierarchy for these disclosures. The carrying amounts reported in a balance sheet for receivables and current liabilities qualify as financial instruments and are a reasonable estimate of their fair values because of the short period of time between their origination and their expected realization and their current market rate of interest. The three levels of valuation hierarchy for fair value determinations are defined as follows: Level 1 inputs include quoted prices for identical assets or liabilities in active markets. Level 2 inputs include observable quoted prices for similar assets and liabilities in active markets, and quoted prices for identical assets or liabilities in inactive markets. Level 3 inputs include one or more unobservable inputs which we have assessed and assumed that market participants would use in pricing the asset or liability. 20 Table of Contents Revenue Recognition Revenue is recognized in the period the services are provided over the contract period, normally one (1) to three (3) years. We invoice one-time startup and implementation costs, such as consolidating and uploading digital assets of the customer, upon completion of those services as one performance obligation and recorded as revenue when completed. Monthly services, such as internet access to software as a service (SaaS), hosting and weekly backups are invoiced monthly as another performance obligation and recorded as revenue over time. Company Recognizes Contract Liability for Its Performance Obligation -- Upon receipt of a prepayment from a customer, the Company recognizes acontract liability in the amount of the prepayment for its performance obligation to transfer goods and services in the future. When the Company transfers those goods and services and, therefore, satisfies its performance obligation to the customer, the Company will then recognize the revenue. Stock-Based Compensation We record stock-based compensation in accordance with Financial Accounting Standards Board FASB Accounting Standards Codification ASC Topic 718, which requires us to measure the cost for any stock-based employee compensation at fair value and recognize the expense over the related service period. We recognize the fair value of stock options, warrants, and any other equity-based compensation issued to employees and non-employees as of the grant date. We use the Black Scholes model to measure the fair value of options and warrants. Income Taxes We account for income taxes in accordance with the asset and liability method of accounting for income taxes, whereby any deferred tax assets are recognized for deductible temporary differences and any deferred tax liabilities are recognized for taxable temporary differences. Deferred tax assets are reduced by a valuation allowance when, in the opinion of our management, it is more likely than not that some portion or all of the deferred tax assets will not be realized. 21 Table of Contents Long-Lived Assets We evaluate the recoverability of our identifiable intangible assets and other long-lived assets when events or circumstances indicate a potential impairment exists. In determining if an impairment exists, we estimate the undiscounted cash flows to be generated from the use and ultimate disposition of these assets. If impairment is indicated based on a comparison of the assets carrying values and the undiscounted cash flows, the impairment loss is measured as the amount by which the carrying amount of the assets exceeds their fair value. Basic and Diluted Earnings Per Share Earnings per share is calculated in accordance with ASC Topic 260, which provides that basic earnings per share is based on the weighted average number of common shares outstanding, and diluted earnings per share is based on the assumption that all dilutive convertible shares, options, and warrants were exercised. Dilution is computed by applying the treasury stock method, which provides that options and warrants are assumed to be exercised at the beginning of the period (or at the time of issuance, if later), and as if the funds obtained thereby are used to purchase common stock at the average market price during the period. Recently Issued Accounting Pronouncements Recent accounting pronouncements issued by the FASB, the AICPA, and the Securities and Exchange Commission did not or are not believed by management to have a material impact on the Company s present or future financial statements. Results of Operations Fiscal Years Ended December 31, 2021 and December 31, 2020 Revenue Revenue was 620,496 for the fiscal year ended December 31, 2021 compared to revenue of 361,888 for the fiscal year endedDecember 31, 2020, an increase of 258,608, or 71 , was due primarily to the Company s acquisition of Score in 2021. Cost of Sales Cost of sales in fiscal year 2021 was 157,991 (25.5 of revenue) compared to cost of sales in fiscal year 2020 of 87,361 (24.1 of revenue), which increase was due primarily due to internal hosting of the Score platform. Gross Margin Gross margin for fiscal year 2021 was 462,505 compared to 274,527 for fiscal year 2020, an increase of 187,978 attributableprimarily to the addition of Score revenue in 2021. Gross margin as a percentage of revenue was 74.5 for fiscal year 2021 compared to 75.9 for fiscal year 2020. Operating Expenses Operating expenses totaled 1,692,534 for fiscal year 2021 compared to 1,467,531 for fiscal year 2020, which comparable operating expenses included: (i) Sales and marketing expenses of 23,938 in 2021 compared to 5,474 in 2020, which large increase of 18,463 in 2021 was due primarily to costs related to the Score acquisition; (ii) Research and development expenses of 452,493 in 2021 compared to 340,232 in 2020, which increased due to development costs of our surgery center business; and (iii) General and administrative expenses of 1,216,103 in 2021 compared to 1,121,825 in 2020, which increased due to an increase in amortization of intangible assets. 22 Table of Contents Operating loss Operating loss was (1,230,029) in 2021 compared to (1,193,004) in 2020. This decrease in the loss was primarily due to the increase in revenue offset by increases in sales and marketing and development expenses. Other Income (Expenses) Other Income (Expenses) for fiscal year 2021 were 4,059,338, including, debt settlement income of 1,439,079, change in fair value of derivatives of 2,806,495, gain on settlement of notes receivable of 555,928, and other items totaling 209,044. These incomes were offset by loan interest of (263,119), loss on change in fair value of Scoreinc.com contingent liability of (75,698), and amortization of debtdiscount of (612,391). In comparison, Other Income (Expenses) for fiscal year 2020 were (1,298,529), including loan interest of (738,443), debtamortization and discount expenses of (318,694), debt settlement and bad debt expenses of (88,740), change in fair value of derivatives of (195,682) and miscellaneous debt expense of (5,228), offset by Other Income of 48,258. The large difference between fiscal 2021 and fiscal 2020 in our Other Income (Expenses) was due primarily to non-cash accounting adjustments related to accounting for our convertible debt and various settlements in the Company s favor. Net Income (Loss) Our net income for fiscal year ended December 31, 2021 was 2,829,309 compared to a net (loss) of (2,491,533) for fiscal year ended December31, 2020. This favorable change of 5,320,842 in fiscal 2021 was due primarily to the various changes and Other Income (Expenses) above described. Liquidity and Capital Resources Our financial condition and future prospects critically depend on our access to financing in order to continue funding our operations. Much of our cost structure is based on costs related to personnel and facilities and is not subject to significant variability. In order to fund our operations and working capital needs, we have historically utilized loans from accredited investors (including management), sales of our common stock and convertible debt securities to accredited investors, and issuances of common stock to satisfy outstanding debt and to pay for development, marketing, management, financial, professional and other services. In order to attain material growth of our SaaS Fision and ScoreCEO platforms and progress with our surgery center project, we will need to raise substantial additional capital through private or public offerings of equity or debt securities, or a combination thereof, and we may have to use a material portion of the capital raised to repay past due debt obligations. To the extent any capital raised is insufficient to satisfy operational working capital needs and meet any required debt payments, we will need to either extend, refinance or convert to equity our past due indebtedness, which there is no assurance we can accomplish. At December 31, 2021 the Company had notes payable indebtedness totaling 2,400,846 including related party accrued interest and debt discounts. Certain information on our notes payable is set forth in Note 8 of the audited consolidated financial statements included in this annual report. In order to fund and conduct our business over the past few years, we relied significantly on working capital obtained from private sales of our equity and convertible debt securities to various accredited investors. Due primarily to our continued substantial operating losses for the past several years, we recently have been unable to continue raising such working capital as needed to support adequately our business plan for future growth. And unless we are able to raise needed substantial additional funding to achieve significant future revenue growth, our current business model most likely will not succeed. The board is exploring raising more working capital through the issuance of common and preferred stock. We have engaged an investment banking firm that specializes in health care to advise on the appropriate structure and to assist raising capital. 23 Table of Contents We may not be able to sell sufficient securities or otherwise obtain such financing when needed on terms acceptable to us, if at all. If further financing is not available, we may be forced to abandon certain business plans or even our entire business. Moreover, regarding any financing we may obtain, any equity or convertible debt financing would be dilutive to our shareholders, and any available debt financing may involve restrictive covenants. Liquidity represents the ability of a company to generate sufficient cash to provide for its immediate needs for cash, which our continued losses have made it difficult for us to satisfy. As of December 31, 2021, we had 478,615 of cash and receivables, and a working capital deficiency of (1,970,604). Over the past few years we have continued to incur substantial losses without any material increase in liquid assets, which has caused a serious and harmful effect to our liquidity and a substantial strain on our ongoing business operations. Along with our limited revenues, we have financed our operations to date through (i) loans from management and from financial and other lenders, including convertible debt (ii) stock-based compensation issued to employees and for consulting, outsourced software, and professional services, (iii) common stock issued to satisfy outstanding loans and accounts payable/accrued expenses, and (iv) equity sales of our common stock. Net Cash Used In Operating Activities We used 944,856 of net cash in operating activities for the fiscal year ended December 31, 2021compared to 229,571 of net cash used in operating activities for the fiscal year ended December 31, 2020. The increase in cash used for operating activities was due primarily increased sales and marketing costs and development and support costs. Net Cash Used In Investing Activities During fiscal years ended December 31, 2021, we spent 126,765 in construction costs for our surgery center project and collected 567,993 on the Continuity Logic receivable. We used no cash flows from investing activities in 2020. Net Cash Provided by Financing Activities During fiscal year ended December 31, 2021, we were provided by financing activities with net cashof 866,110 including proceeds from sales of common stock of 216,110 and issuance of convertible notes payable of 780,271, offset by the repayment of notes payable of 104,771 and repayments of related party notes payable of 25,500. In comparison, during fiscal year ended December 31, 2020, we were provided by financing activities with net cashof 222,565 including proceeds from sales of common stock of 50,000, proceeds from an SBA-PPP loan of 177,200, and proceeds from a Note Payable of 78,000, offset by repayments on Notes Payable and a line of credit totaling of 82,635. Convertible Note Financing For the past few years, a majority of our financing has consisted of convertible notes sold to accredited investors in private transactions. In 2021 and 2020, we raised 780,271 and 78,000, respectively, through issuance of convertible notes. Our 2018-2019 Private Placement of Securities In October 2018 we commenced a private offering of 2,000,000 of our common stock at .20 per share (10,000,000 shares), offered only to accredited investors. All private investors in this placement also received additional Advisory Shares, Warrants and potential True-Up and Penalty Shares as follows: Advisory Shares Each investor also received an amount of Advisory Shares equal to 10 of the common shares purchased by the investor at .20 per share. Warrants Each private investor also received three-year warrants to purchase additional common shares equal to their purchased shares at an exercise price of .20 per share. True-Up and Penalty Shares Each private investor also received the right to receive True-Up shares in the event the True-Up Price as defined in the Stock Purchase Agreement for this placement is lower than the .20 price for the purchased shares. The True-Up Price represents the average of the fifteen lowest public market closing prices during a specified 90-day period and having a floor of .10 per share. Based on trading of our common stock, the calculated True-Up Price per share relating to such a 15-day average became lower than this .10 floor. 24 Table of Contents In addition, the warrants issued to investors in this private offering are subject to the same True-Up Price adjustment regarding their exercise price. In 2019, we sold 8,750,000 common shares offered in this private placement and received total proceeds of 1,750,000 from investors, for which we issued an aggregate of 8,750,000 purchased shares and 875,000 Advisory Shares. In 2020, we issued an additional total of 9,625,000 common shares to investors in this private offering for required true-up and penalty shares. Going Concern As stated in the audited financial statements included in this Annual Report on Form 10-K, these financial statements have been prepared on a going concern basis, which contemplates and implies that the Company will continue to realize its assets and satisfy its liabilities and commitments in the normal course of business. For the years ended December 31, 2021 and 2020, we incurred operating losses of (1,230,029) and (1,193,004), respectively. We had an accumulated deficit of (31,474,016) and a working capital deficiency of (1,970,604) as of December 31, 2021. We are continuing to incur material losses in 2022. These adverse financial conditions raise substantial doubt about our ability to continue as a going concern. Our financial statements do not include any adjustments relating to the recoverability and classification of recorded asset amounts, or the amounts and classification of liabilities that might be necessary if we are unable to continue as a going concern. Off-Balance Sheet Arrangements We have no off-balance sheet items as of December 31, 2021 or 2020. ITEM 7A QUANTITATIVE AND QUALITATIVE DISCLOSURES ABOUT MARKET RISK. Not applicable. ITEM 8 FINANCIAL STATEMENTS AND SUPPLEMENTARY DATA. See our financial statements beginning on page F-1 of this Annual Report. ITEM 9 CHANGES IN AND DISAGREEMENTS WITH ACCOUNTANTS ON ACCOUNTING AND FINANCIAL DISCLOSURE. None. ITEM 9A CONTROLS AND PROCEDURES Evaluation of Disclosure Controls and Procedures We maintain disclosure controls and procedures that are designed to ensure that information required to be disclosed in our reports filed pursuant to the Securities Exchange Act of 1934 Exchange Act is recorded, processed, summarized and reported within the time periods specified in the rules and forms of the SEC and that such information is accumulated and communicated to our management, including our Chief Executive Officer and our Chief Financial Officer, as appropriate, to allow for timely decisions regarding required disclosure. In designing and evaluating these disclosure controls and procedures, our management recognizes that any controls and procedures, no matter how well designed and operated, can provide only reasonable assurance of achieving their desired objectives. Our management will apply its best judgment in evaluating the cost-benefit relationship of any disclosure controls and procedures adopted by us. 25 Table of Contents Because of its inherent limitations, internal control over financial reporting may not prevent or detect misstatements. Moreover, projections of any evaluation of effectiveness to future periods are subject to the risk that controls may become inadequate due to changes in conditions, or that the degree of compliance with the policies and procedures may deteriorate. The design of our disclosure controls and procedures must reflect the fact that we will face personnel and financial restraints for some time, and accordingly the benefit of such controls must be considered relative to their costs. Management s Report on Internal Control Over Financial Reporting Our management is responsible for establishing and maintaining adequate internal control over financial reporting (as defined in Rules 13a-15(f) and 15d-15(f) under the Exchange Act). Our internal control over financial reporting is designed to provide reasonable assurance regarding the reliability of our financial reporting and the preparation of our financial statements for external purposes in accordance with generally accepted accounting principles. Our internal control over financial reporting is being designed to include policies and procedures that are intended to: (i) maintain records that in reasonable detail accurately and fairly reflect the transactions and dispositions of our assets; (ii) provide reasonable assurance that transactions are recorded as necessary to permit preparation of financial statements in accordance with U.S. GAAP, and that our receipts and expenditures are being made only in accordance with the authorization of our management and directors; and provide reasonable assurance regarding prevention or timely detection of unauthorized acquisition, use or disposition of our assets that could have a material effect on our financial statements. Our management assessed the effectiveness of our internal control over financial reporting as of December 31, 2021. In making this assessment, our management applied the integrated framework and criteria which has been developed and set forth by the Committee of Sponsoring Organizations of the Treadway Commission (COSO) (revised 2013). This assessment included an evaluation of the design and procedures of our control over financial reporting. Based on this evaluation, our management concluded that as of December 31, 2021 our internal control over financial reporting was not effective due to certain material weaknesses. These identified material weaknesses included (i) an insufficient accounting staff; and (ii) limited checks and balances in processing cash and other transactions. Changes in Internal Control over Financial Reporting There have been no changes in our internal control over financial reporting during our last fiscal year ended December 31, 2021 that have materially affected, or are reasonably likely to materially affect, our internal control over financial reporting. ITEM 9B OTHER INFORMATION None. 26 Table of Contents PART III ITEM 10 DIRECTORS, EXECUTIVE OFFICERS AND CORPORATE GOVERNANCE. The following sets forth information about our directors and executive officers: Name Age Position Michael P. Brown 63 Director and Chairman of Board Greg Nagel 65 Director John Bode 48 Director and Interim Chief Executive Officer William Gerhauser 53 Director Joshua Carmona 57 Director and Chief Technology Officer Michael P. Brown, a founder of the Company, has been a director and Chairman of the Board of the Company since our inception in 2010, and he was the Chief Executive Officer of the Company from our inception until September 2019. Mr. Brown has been an accomplished corporate executive for more than 20 years while serving in various senior operations, sales and executive positions. His extensive senior management experience includes having been Senior Vice President of Operations at Life Time Fitness from 1997 to 2007, where he successfully managed and oversaw annual revenue growth from 7 Million to 689 Million. In 2001, he attended the Executive MBA course at the University of St. Thomas, and he has also served in the United States Navy, Submarine Service. Greg Nagel has been an independent director of the Company since March 2020. Mr. Nagel has over 20 years of experience leading large sales and marketing teams in the surgical distribution space. For the past six years, Mr. Nagel has been a distributor for Innovative Sterilization Technologies, Inc. Mr. Nagel is a graduate of Augustana University. John Bode has been an independent director of the Company since March 2018 and was appointed interim CEO on September 18,2022. Mr. Bode s past corporate experience includes many years serving as a key executive and/or financial officer for leading public companies. Since February 2015, he has been the owner and managing director of Aerie Investments, LLC, an investment company focused on assisting digital media start-ups with business development, strategic initiatives, and raising capital. Prior thereto, Mr. Bode was Chief Financial Officer (CFO) of the Tribune Publishing Company (now tronc, Inc, (NASDAQ: TRNC) from October 2013 to January 2015. Mr. Bode also served as the CFO of Source Interlink Companies, one of the largest enthusiast media companies in the USA and a leading distributor of periodicals. His extensive past accounting/auditing career also included being employed as a Certified Public Accountant (CPA) for BDO Seidman. William Gerhauser has been a director of the Company since April 2020. Mr. Gerhauser has over 25 years of experience in the financial industry in both the United States and Europe. For the past five years, Mr. Gerhauser is the Chief Executive Officer of affiliate Capital Market Solutions LLC ("CMS"). CMS is an international consultancy firm that provides accounting, legal, structural and strategic advice to several private and public companies. Mr. Gerhauser is also the President of MGA Holdings, LLC, and also the Managing Partner of Ignition Capital, LLC. Mr. Gerhauser is also a co-founder of Fighters Choice, Inc. Mr. Gerhauser s specialty is niche markets that are heavily regulated or have unusual tax considerations. In the last 15 years he has successfully initiated transactions in the Oil and Gas, Entertainment, Biotech, Information Technology and Beverage Industries. He is a UK-US dual national. He is a graduate of The Royal College of St. Peter at Westminster, and is a U.S. Army Infantry Veteran. Joshua Carmona has been a director of the Company since May 2021 upon the Company s purchase acquisition of Score, Inc. Mr. Carmona is the Chief Technology Officer of the Company and has a combined three decades of experience in Technology, Automotive Executive Operations Management, Enterprise Sales, Enterprise Customer service, Business Coaching, and Enterprise Level Credit Experience. He was responsible for creating ScoreInc.com s World Class Dispute Process, Founder and President of ScoreInc.com, Co-founder of the Credit Repair Summit, Co-creator of ScoreWay University, Architect of ScoreCEO.com and consultant to thousands of credit repair businesses in the United States, Puerto Rico, and Canada. Mr. Carmona entered the Hemp industry as the Chief Operations Officer of Xtraction Services in 2017 which later went public on the Canadian Stock Exchange. As an Infusion Elite Forum and Disney Institute (World Class Customer Service, Toyota Excellence Program) Graduate, US Army Infantry Officer, Joe Verde Group Graduate, Microsoft Business Consultant, IBM Best Team Engineer, and his Industrial Engineering background Mr. Carmona has been able to leverage his can-do attitude, education, and experience to successfully drive revenue and increase profitability in the past. Director Relationships There are no family relationships among or between any of our officers and directors. None of our directors is a director of any other company having a class of securities registered pursuant to Section 12 of the Securities Exchange Act or subject to the requirements of Section 15(d) of the Securities Exchange Act, or of any company registered as an investment company under the Investment Company Act of 1940. 27 Table of Contents Involvement in Certain Legal Proceedings To the best of our knowledge, our directors and executive officers have not been involved in any of the following legal proceedings during the past five years: i) any bankruptcy petition filed by or against such person or any business of which such person was a general partner or executive officer either at the time of the bankruptcy or within two years prior to that time; ii) any conviction in a criminal proceeding or being subject to a pending criminal proceeding (excluding traffic violations and other minor offenses); iii) being subject to any order, judgment, or decree, not subsequently reversed, suspended, or vacated, of any court of competent jurisdiction, permanently or temporarily enjoining or otherwise limiting the involvement of the person from any type of business, securities or banking activities or to be associated with any person practicing in banking or securities activities; iv) being found by a court of competent jurisdiction in a civil action, the SEC or the Commodity Futures Trading Commission to have violated a federal or state securities or commodities law, and the judgment has not been reversed, suspended, or vacated; v) being subject of, or a party to, a federal or state judicial or administrative order, judgment, or decree or finding, not subsequently suspended, reversed or vacated, relating to an alleged violation of any federal or state securities or commodities laws or regulation, any law or regulation respecting financial institutions or insurance companies, or any law or regulation prohibiting mail or wire fraud or fraud in connection with any business entity; or vi) being subject of or party to any sanction or order, not subsequently suspended, reversed or vacated, of any self-regulatory organization, any registered entity or any equivalent exchange, association, entity or organization that has disciplinary authority over its members or persons associated with a member. Director Committees Currently we do not have any designated director committees due to the small size of our Board of Directors. Our entire Board of Directors performs the functions of an audit committee , including approving the annual selection of our independent auditing firm. Section 16(a) Beneficial Ownership Reporting Section 16(a) of the Exchange Act requires our directors and officers and any persons who beneficially own more than ten percent of our common stock to file with the SEC initial reports of ownership and reports of changes in ownership of our common stock. To the best of our knowledge, regarding any Section 16(a) forms with respect to our fiscal year ended December 31, 2021, all such required reports were filed with the SEC. Code of Ethics We have not adopted a Code of Ethics for our management. When we adopt a Code of Ethics, it will apply to all our executive officers and directors as well as any other employees who perform any accounting or financial functions or operations for us. ITEM 11 EXECUTIVE COMPENSATION Summary Compensation Table The following table presents information for our last two fiscal years ended December 31, 2021 and 2020 regarding all compensation from us paid to, awarded to, and earned by each individual who served as our chief executive officer or in any other executive officer position with the Company. Messrs. Laurence Mascera and Daniel Dorsey received no compensation directly from us for their respective executive officer roles with the Company in early 2020, when Mr. Mascera served as our interim Chief Executive Officer until his resignation on March 31, 2020, and Mr. Dorsey served as our Chief Financial Officer until his resignation on March 31, 2020. Through March 2020, such executive management services were included as part of the overall monthly consulting services provided to us by our affiliate Capital Markets Solutions, LLC (CMS), for which services we paid 50,000 monthly. 28 Table of Contents Option Stock Name and Position Year Salary Bonus Awards Awards Total Michael P. Brown (1) 2021 - - - 50,000 50,000 Chief Executive Officer and Director 2020 - - - - - William Gerhauser (2) 2021 - - - 50,000 50,000 Chief Executive Officer and Director 2020 - - - - - (1) Mr. Brown acted as Principal Executive and Financial Officer of the Company from September 2019 through November 19, 2021. (2) Mr. Gerhauser was appointed Chief Executive Officer by the Board of Directors on November 19, 2021. Executive Compensation Overview Our executive compensation program historically has consisted of a combination of base salary and stock-based compensation in the form of common stock awards or options. Our executive officers and any other salaried employees are also eligible to receive any health and other fringe benefits offered by the Company. Employment Agreements With Our Executive Officers We currently have no outstanding employment agreements with any executive officers. Equity Compensation Plans Our 2011 Plan In 2011 the Board of Directors and shareholders of our Minnesota Fision subsidiary adopted and approved our 2011 Stock Option and Compensation Plan (the Plan ), which as amended is our only equity compensation plan approved by stockholders. The purpose of the Plan is to advance the interests of the Company and our stockholders by attracting, retaining and rewarding our employees and key consultants performing services for us, and to motivate them to contribute to our growth and profitability. Issuance of Awards. Under the Plan, the issuance of awards and the persons, terms and conditions regarding any award are determined by our Board of Directors. Including amendments to the Plan, we have reserved up to 4,600,000 shares of our common stock to satisfy any awards under the Plan which can include one or a combination of stock options, stock appreciation rights (SARs), stock awards, restricted stock, performance shares, or cash. 29 Table of Contents To date, we have only granted stock option awards under the Plan. As of December 31, 2021 and 2020, all of the outstanding options under the Plan expired, and since none had been exercised according to their terms, their underlying shares were returned to the Plan for future availability. Term and Vesting of Options. Incident to granting an option award, the Board of Directors must fix the term, exercise price, number of optionshares, and any applicable vesting conditions or schedule. The Board of Directors retains the right to accelerate vesting of an option anytime for any reason. The maximum term of an option under the Plan is ten years. Exercise Price and Manner of Exercise. The exercise price for any option granted under the Plan is determined by the Board of Directors, providedthat the option price for any Incentive Stock Options intended to qualify under Section 422A of the Internal Revenue Code of 1986 shall not be less than the Fair Market Value of our common stock as of the date of grant. The exercise price may be paid in cash or check, cashless exercise, or tender of our shares already owned by the option holder, with any cashless exercise or tender of shares being based on the fair market value of our common shares on the date of exercise. Transferability of Awards. Grants of options under the Plan are not transferable other than by will or laws of descent and can only be exercised bythe grantee during his or her lifetime. Immediate Acceleration. The Plan provides that all options and awards not yet vested will be fully vested, and all performance conditions forrestricted stock or performance share awards shall be deemed satisfied, in the event of (i) a change of control whereby a person or related group obtains 40 or more of our common stock, or (ii) a majority of our directors is replaced within 24 months without approval of the existing Board of Directors, or (iii) our stockholders approve a merger or the sale or other disposition of substantially all our assets. Outstanding Equity Awards at Fiscal Year End The Company has no outstanding equity awards as of December 31, 2021. Director Compensation For the year ended December 31, 2020, our only director compensation was paid to our two independent directors, John Bode and Gregory Nagel, who each receives 12,500 in value of our common shares quarterly for serving as an independent director. Mr. Bode received a total of 6,783,631 common shares for his services as a director in 2020 and Mr. Nagel received a total of 5,191,028 common shares for his services as a director in 2020. For the year ended December 31, 2021, our director compensation was paid in the form of restricted shares. Each director received 12,500 in value of our common shares quarterly for serving as director. William Gerhauser, Greg Nagel, and Michael Brown each received 1,939,742 restricted shares of common stock valued at 50,000 for 2021 service. John Bode received 1,950,809 restricted shares of common stock valued at 50,000 for 2021 service. Josh Carmona received 1,094,425 restricted shares of common stock valued at 25,000 for 2021 service. ITEM 12 SECURITY OWNERSHIP OF CERTAIN BENEFICIAL OWNERS AND MANAGEMENT AND RELATED STOCKHOLDER MATTERS. The following table sets forth certain information regarding the beneficial ownership of common stock of the Company as of December 31, 2021, by each of our directors and executive officers, each person known by us to own beneficially 5 or more of our Common Stock, and all directors and executive officers as a group. The amounts and percentages of common stock beneficially owned in this table are reported on the basis of regulations of the Securities and Exchange Commission. To the best of our knowledge, each beneficial owner named in this table has sole voting and sole investment power with respect to all shares beneficially owned. The address of each beneficial owner is c/o Fision Holdings, Inc., 1650 West End Boulevard, Suite 100, Minneapolis, MN 55416. Shares of Percent of Common Common Name Stock Stock (1) Michael P. Brown 27,000,625 5.88 John Bode 10,633,508 2.29 William Gerhauser (2) 81,593,209 17.77 Gregory Nagel 7,130,770 1.55 Joshua Carmona 1,094,425 0.24 All directors and officers as a group (five persons) 127,320,013 27.73 Capital Market Solutions, LLC CMS (3) 93,363,145 20.33 Daniel Mangless 28,479,096 6.20 Thomas Bibb (4) 37,863,179 8.25 (1) Based on 459,128,589 outstanding common shares of the Company as of December 31, 2021. (2) Includes 93,363,145 shares (28,291,122 shares and 35,072,023 warrants) owned by affiliate CMS which is controlled by Mr. Gerhauser; 8,008,655 shares (5,509,733 shares and 2,498,922 warrants) owned by affiliate Ignition Capital LLC, 3,281,667 shares owned by affiliate MGA Holdings LLC, and 5,000,000 shares owned by affiliate Fighters Choice Management LLC, all of which are controlled by Mr. Gerhauser. (3) Includes 35,072,023 shares underlying warrants having a term of five years. (4) Includes 7,155,405 shares owned by Joy Pipe USA LP and 2,500,000 shares owned by Bibb Investments, LLC, which are both controlled by Mr. Bibb 30 Table of Contents ITEM 13 CERTAIN RELATIONSHIPS AND RELATED TRANSACTIONS, AND DIRECTOR INDEPENDENCE Transactions With Related Parties Since January 1, 2020 we have been involved in the following transactions with related parties: In recent years, the Company has been unable to compensate its principal officers in cash under the terms of their respective employment agreements, and as a result their accrued compensation from time to time was converted into notes payable bearing interest at 6 per annum. These Notes in turn have been converted into restricted common stock of the Company as follows: in December 2020, Messrs. Brown and Lowenthal converted a total of 350,000 of their Notes into a total of 7,000,000 common shares at .05 per share including 5,000,000 shares for Mr. Brown and 2,000,000 shares for Mr. Lowenthal; and in January 2021, Messrs. Brown and Lowenthal converted all remaining balances of their Notes totaling 442,103.62 into a total of 8,842,072 shares at .05 per share including 5,084,535 shares for Mr. Brown and 3,757,537 shares for Mr. Lowenthal. Effective April 1, 2019 we entered into a one-year Consulting Agreement with our affiliate, Capital Market Solutions, LLC CMS ). CMS is controlled by William Gerhauser, who is a director of the Company. During the one-year term of this consulting agreement, CMS provided certain management, investor and public relations services, and other advisory services to us in consideration for 50,000 monthly, 30 million restricted shares of our common stock, and a five-year stock purchase warrant to purchase an additional 30 million common shares at .20 per share. CMS provided management services to us through March 2020 including providing Laurence Mascera as our Chief Executive Officer, and Daniel Dorsey as our Chief Financial Officer. Effective March 31, 2020, Messrs. Mascera and Dorsey resigned all respective management positions and directorships with the Company. Effective March 31, 2020, Michael Brown assumed the interim executive positions of principal executive officer and principal financial officer of the Company. Effective November 19, 2021, the Board of Directors approved William Gerhauser, a director, the Chief Executive Officer of the Company. CMS has additional short term notes for a total of 76,111 as of December 31, 2020, which have been in default since 2019. Also, an additional short-term note for 82,500 with affiliate MGA Capital LLC and two additional short-term notes for an aggregate of 148,778 with affiliate Ignition Capital, LLC are also in default, as of December 31, 2020, whereby MGA Capital and Ignition Capital are partially owned by our director William Gerhauser. CMS, MGA Capital, and Ignition Capital all converted these short-term notes payable into common stock during 2021. Incident to the note conversions during 2021, certain related party noteholders converted an aggregate of 732,525 of their Notes and accrued interest into 18,167,057 shares of our restricted common stock. Incident to the note conversions during 2020 described above, certain related party noteholders converted an aggregate of 750,000 of their Notes and accrued interest into 15,000,000 shares of our restricted common stock. During the year ended December 31, 2021 and 2020, we issued a total of 8,864,200 and 11,974,659 shares of our restricted common stock to our directors for management services. 31 Table of Contents In November 2020, the Company acquired two Florida LLCs from affiliate CMS which are engaged in the business of developing: i) a medical Ambulatory Surgery Center in Ft Myers FL and ii) related software applications for operation of this Ft Myers facility and other medical AmbulatorySurgery Centers. For a description of this acquisition, see Recent Acquisition to Engage in Medical Ambulatory Surgery Center Business included in Item 1 of this Annual Report. Related Party Transaction Policy Our Board of Directors has no formal written policy regarding related-party transactions, but rather follows the requirements of state law applicable to such transactions. In particular, after full disclosure of all material facts, review and discussion, our Board of Directors approves or disapproves such transactions. No director is allowed to vote for approval of a related-party transaction for which he or she is the related party, but shall provide all material information concerning the related-party transaction and the director s interest in the transaction. Director Independence As of the date of this Annual report, Greg Nagel is the only independent directors of the Company. ITEM 14 PRINCIPAL ACCOUNTING FEES AND SERVICES The following table includes all fees billed for auditing and any other services provided to us by Assurance Dimensions, certified public accountants associates, for the fiscal years ended December 31, 2021 and 2020. 2021 2020 Audit Fees 84,500 56,500 Audit-Related Fees 32,500 - Tax Fees - - All Other Fees - - Total 117,000 56,500 Our Board of Directors has adopted a procedure for pre-approval of all fees charged by our independent registered public accounting firm. Under this procedure, our Board of Directors approves the annual engagement letter with respect to audit and review services. Any other fees are subject to pre-approval by our Board of Directors. The audit fees paid to our auditor with respect to 2021 and 2020 were pre-approved by our entire Board of Directors. 32 Table of Contents PART IV ITEM 15 EXHIBITS, FINANCIAL STATEMENTS, SCHEDULES FINANCIAL STATEMENTS Item Page Report of Independent Registered Public Accounting Firm F-1 Consolidated Balance Sheets December 31, 2021 and 2020 F-3 Consolidated Statements of Operations Years Ended December 31, 2021 and 2020 F-4 Consolidated Statements of Shareholders Deficit Years Ended December 31, 2021 and 2020 F-5 Consolidated Statements of Cash Flows Years Ended December 31, 2021 and 2020 F-6 Notes to Consolidated Financial Statements F-7 33 Table of Contents REPORT OF INDEPENDENT REGISTERED PUBLIC ACCOUNTING FIRM To the Board of Directors and Stockholders of Fision Corporation Opinion on the Financial Statements We have audited the accompanying consolidated balance sheet of Fision Corporation (the Company) as of December 31, 2021 and 2020, and the related consolidated statements of operations, stockholders deficit and cash flows for each of the years in the two-year period ended December 31, 2021 and the related notes (collectively referred to as the consolidated financial statements ). In our opinion, the consolidated financial statements present fairly, in all material respects, the financial position of the Company as of December 31, 2021 and 2020, and the results of its operations and its cash flows for each of the years in the two-year period ended December 31, 2021, in conformity with accounting principles generally accepted in the United States of America. Explanatory Paragraph- Going Concern The accompanying consolidated financial statements have been prepared assuming that the Company will continue as a going concern. As discussed in Note 2 to the financial statements, the Company has suffered recurring operating losses for the year ended December 31, 2021. The Company had an operating loss of 1,230,029, accumulated deficit of 31,474,016, net cash used in operating activities of 944,856, and had negative working capital of 1,970,604. These factors raise substantial doubt about the Company s ability to continue as a going concern. Management s plans in regard to these matters are also described in Note 2. The consolidated financial statements do not include any adjustments that might result from the outcome of this uncertainty Basis for Opinion These consolidated financial statements are the responsibility of the Company s management. Our responsibility is to express an opinion on the Company s consolidated financial statements based on our audits. We are a public accounting firm registered with the Public Company Accounting Oversight Board (United States) (PCAOB) and are required to be independent with respect to the Company in accordance with the U.S. federal securities laws and the applicable rules and regulations of the Securities and Exchange Commission and the PCAOB. We conducted our audits in accordance with the standards of the PCAOB. Those standards require that we plan and perform the audit to obtain reasonable assurance about whether the consolidated financial statements are free of material misstatement, whether due to error or fraud. The Company is not required to have, nor were we engaged to perform, an audit of its internal control over financial reporting. As part of our audits, we are required to obtain an understanding of internal control over financial reporting, but not for the purpose of expressing an opinion on the effectiveness of the Company s internal control over financial reporting. Accordingly, we express no such opinion. Our audits included performing procedures to assess the risks of material misstatement of the consolidated financial statements, whether due to error or fraud, and performing procedures that respond to those risks. Such procedures included examining, on a test basis, evidence regarding the amounts and disclosures in the consolidated financial statements. Our audits also included evaluating the accounting principles used and significant estimates made by management, as well as evaluating the overall presentation of the consolidated financial statements. We believe that our audits provide a reasonable basis for our opinion. F-1 Table of Contents Critical Audit Matters The critical audit matters communicated below are matters arising from the current period audit of the financial statements that were communicated or required to be communicated to the audit committee and that: (1) relate to accounts or disclosures that are material to the financial statements and (2) involved our especially challenging, subjective, or complex judgments. The communication of critical audit matters does not alter in any way our opinion on the financial statements, taken as a whole, and we are not, by communicating the critical audit matters below, providing separate opinions on the critical audit matters or on the accounts or disclosures to which they relate. Valuation of Intangible Assets in the Score, Inc. Acquisition As described in Notes 1 and 11 to the consolidated financial statements, the Company completed the acquisition of Score, Inc. (Score) for consideration of 789,370 and the transaction was accounted for as a business combination. The acquired intangible assets included developed technology, permits, website, partner agreement, customer relationship, trade name and customer relationships valued at a total of approximately 663,000. In addition goodwill of approximately 174,000 was recorded. The Company recorded the acquired intangible assets at fair value on the date of acquisition using a discounted cash flow methodology to developed technology and trade name and a cost approach to permits, website and partner agreement. The methods used to estimate the fair value of acquired intangible assets involve significant assumptions. The significant assumptions applied by management in estimating the fair value of acquired intangible assets included income projections and discount rates. The principal considerations for our determination that performing procedures relating to the valuation of intangible assets in the Score acquisition is a critical audit matter are (1) there was a high degree of auditor judgment and subjectivity in applying procedures relating to the fair value of intangible assets acquired due to the significant judgment by management when developing the estimates and (2) significant audit effort was required in evaluating the significant assumptions relating to the estimates, including the income projections and discount rates. In addition, the audit effort involved the use of professionals with specialized skill and knowledge to assist in performing these procedures and evaluating the audit evidence obtained. Addressing the matter involved performing procedures and evaluating audit evidence in connection with forming our overall opinion on the consolidated financial statements. These procedures included, among others, reading the memorandum of understanding, and testing management s process for estimating the fair value of intangible assets. Testing management s process included evaluating the appropriateness of the valuation models, testing the completeness, accuracy, and relevance of underlying data used in the models, and testing the reasonableness of significant assumptions, including the income projections and discount rates. Evaluating the reasonableness of the income projections involved considering the current performance of the acquired business, the consistency with external market and industry data, and whether these assumptions were consistent with other evidence obtained in other areas of the audit. Professionals with specialized skill and knowledge were used to assist in evaluating the reasonableness of significant assumptions, including the discount rates, by comparing them against discount rate ranges that were independently developed using publicly available market data for comparable companies. We have served as the Company s auditor since 2019. December 30, 2022 PCAOB ID F-2 Table of Contents FISION CORPORATION CONSOLIDATED BALANCE SHEET December 31, December 31, 2021 2020 ASSETS Current Assets: Cash Accounts receivable, net Notes Receivable, net of allowance for doubtful accounts (Note 5) Total Current Assets Property, plant and equipment Other Assets: Goodwill Intangible assets, net of accumulated amortization Deposits Total Assets LIABILITIES AND STOCKHOLDERS' DEFICIT Current Liabilities: Accounts payable, accrued expenses and other current liabilities Derivative liability Current portion of long-term debt: SBA/PPP Loan Notes payable and accrued interest - related party, net of debt discount of and respectively Notes payable, net of debt discount of and respectively Total Current Liabilities Long-Term Liabilities: Long-Term SBA-PPP loan Contingent liability Scoreinc.com acquisition Long-Term Convertible Notes Payable and accrued interest, related party, net of debt discount of and respectively Long-Term Convertible Notes Payable, net of debt discount of and respectively Total Long-Term Liabilities Total Liabilities Contingencies and Commitments (Note 12) Stockholders' Deficit: Preferred Stock, Par value, shares authorized, no shares issued and outstanding Common Stock, Par value, shares authorized and shares issued and outstanding, respectively Additional paid in capital Accumulated deficit Total Stockholders' Deficit Total Liabilities and Stockholders' Deficit The accompanying notes are an integral part of these audited consolidated financial statements. F-3 Table of Contents FISION CORPORATION CONSOLIDATED STATEMENTS OF OPERATIONS FOR YEARS ENDED DECEMBER 31, 2021 AND 2020 2021 2020 REVENUE COST OF SALES GROSS PROFIT OPERATING EXPENSES Sales and marketing Research and development General and administrative TOTAL OPERATING EXPENSES OPERATING LOSS OTHER INCOME / (EXPENSES) Interest expense Amortization of debt discount Other expenses Gain/(Loss) on fair value of derivatives Gain/(Loss) on extinguishment of debt / settlement of debt Loss on fair value of Scoreinc.com contingent liability Gain/(Loss) on settlement of note receivable SBA-PPP loan forgiveness Other income TOTAL OTHER INCOME (EXPENSES) NET INCOME/(LOSS) Net income/(loss) per common share - basic Weighted average common shares outstanding basic Net income/(loss) per common share - diluted Weighted average common shares outstanding diluted The accompanying notes are an integral part of these audited consolidated financial statements. F-4 Table of Contents FISION CORPORATION CONSOLIDATED STATEMENTS OF STOCKHOLDERS' DEFICIT FOR THE YEARS ENDED DECEMBER 31, 2021 2020 Additional Common Common Paid in Accumulated Shares Stock Capital Deficit Total Balance, December 31, 2019 Common stock issued for cash Stock Issued for True-Up Penalty Shares Stock issued for services Warrants/Options granted for services - Conversion of notes payable and accrued interest Net loss - Balance, December 31, 2020 Stock issued for services Warrants exercised Conversion of notes payable and accrued interest Net income - Balance, December 31, 2021 The accompanying notes are an integral part of these audited consolidated financial statements. F-5 Table of Contents FISION CORPORATION CONSOLIDATED STATEMENTS OF CASH FLOWS FOR THE YEARS ENDED DECEMBER 31, 2021 AND 2020 CASH FLOWS FROM OPERATING ACTIVITIES: 2021 2020 Net Income (Loss) for the Period Net cash used in operating activities: Common stock issued for services Amortization of pre-paid stock expenses Depreciation and amortization True-up and penalty shares issued Stock warrants/stock options issued for services Loss (Gain) in fair value of derivative liabilities Loss on extinguishment of debt/gain on settlement of debt Loss on change in fair value of Score contingent liability Paycheck Protection Program loan forgiveness Impairment and other charges Bad debt expense on note receivable Gain on settlement of notes receivable Amortization of debt discount Changes in Operating Assets and Liabilities (Increase) decrease in: Accounts receivable Other current assets Change in interest receivable Deposits Increase (decrease) in: Accounts payable, accrued expenses and other current liabilities Net Cash Used in Operating Activities CASH FLOWS FROM INVESTING ACTIVITIES: Cash collected on notes receivable Payments made for construction in process Net Cash Used In Investing Activities CASH FLOWS FROM FINANCING ACTIVITIES: Repayments on note payable Proceeds from convertible note payable Proceeds from SBA/PPP loan Repayments for related party notes and line of credit Proceeds from issuance of common stock for warrant exercises Net Cash Provided by Financing Activities Net Increase (Decrease) in Cash Cash at Beginning of Period Cash at End of Period SUPPLEMENTAL DISCLOSURE OF CASH FLOW INFORMATION: Noncash operating and financing activities: Accrued interest increasing notes payable Derivatives recorded as debt discount Acquisition of subsidiary with convertible notes Conversion of debt and accrued interest to common stock Conversion of accrued salaries and accrued expenses to convertible notes payable, related party Conversion of accrued salaries and accrued expenses to convertible notes payable The accompanying notes are an integral part of these audited consolidated financial statements. F-6 Table of Contents FISION CORPORATION NOTES TO CONSOLIDATED FINANCIAL STATEMENTS DECEMBER 31, 2021 AND 2020 Principles of Consolidation Use of estimates Concentration of Credit Risk . As of December 31, 2021, the Company held of cash deposits in excess of federally insured limits. We maintain cash balances at high quality financial institutions to mitigate this risk. We perform ongoing credit evaluations of our customers and generally do not require collateral from them to do business with us. Cash equivalents Accounts Receivable and Allowance for Doubtful Accounts and as of December 31, 2021 and 2020, respectively. F-7 Table of Contents years Computer and office equipment years Lease Accounting Impairment of long-lived assets Goodwill and Other Intangible Assets Revenue recognition Credit repair software solutions Net income (loss) for dilutive earnings per share Company Recognizes Contract Liability for Its Performance Obligation Upon receipt of a prepayment from a customer, the Company recognizes a contract liability in the amount of the prepayment for its performance obligation to transfer goods and services in the future. When the Company transfers those goods and services and, therefore, satisfies its performance obligation to the customer, the Company will then recognize the revenue. F-8 Table of Contents . Section 740-10-25 also provides guidance on de-recognition, classification, interest and penalties on income taxes, accounting in interim periods and requires increased disclosures. We had no material adjustments to our assets and/or liabilities for unrecognized income tax benefits according to the provisions of Section 740-10-25. Stock-Based Compensation Net income (loss) per share Dilutive effect of convertible debt Net income (loss) for dilutive earnings per share Shares used to compute basic earnings per share Dilutive effect of warrants and convertible debt - Shares used to compute dilutive earnings per share Basic earnings per share Diluted earnings per share For the year ended December 31, 2021 and 2020, there were and potentially dilutive securities not included in the calculation of weighted-average common shares outstanding since they would be anti-dilutive. Research and Development for research and development. In comparison, during the fiscal year ended December 31, 2020 we incurred total expenses of for research and development. Such costs related to software development are included in research and development expense until the point that technological feasibility is reached, which for our software products, is generally shortly before the products are released to manufacturing. Once technological feasibility is reached, such costs are capitalized and amortized to cost of revenue over the estimated lives of the products. Advertising Costs and respectively. The costs are included in the sales and marketing expenses on the consolidated statement of operations. F-9 Table of Contents and an accumulated deficit of as of December 31, 2021, and we had an operating loss of for 2021. These conditions raise substantial doubt about our ability to continue as a going concern for one year after issuance date of the financial statements. These financial statements do not include any adjustments relating to the recoverability and classification of recorded asset amounts or amounts and classification of liabilities that might result from this uncertainty. Our ability to continue as a going concern depends on our ability to raise additional significant funds. The Company is attempting to raise such funds through private placements of its equity or debt (including convertible debt) securities, but there can be no assurance any such future funds will become available to us. Unless we can raise additional significant working capital, our business plan most likely will not succeed. The following table represents our assets and liabilities by level measured at fair value on a recurring basis at December 31, 2020. Level 1 Level 2 Level 3 Derivative Liability F-10 Table of Contents Fair Value Change in Fair Value January 1, fair December 31, 2021 Additions Value Conversions 2021 Derivative Liability All gains and losses on assets and liabilities measured at fair value on a recurring basis and classified as Level 3 within the fair value hierarchy are recognized in other interest and expense in the accompanying financial statements. We also recognized a gain on extinguishment of debt of for the year ended December 31, 2021. The derivative liability relating to warrants and the beneficial conversion feature of our convertible notes payable was at December 31, 2021 and was computed using the following variables: -- Expected volatility Expected term Due on demand to Months Risk free interest rate Expected dividends - Derivative Instruments The Company accounts for derivative instruments in accordance with Accounting Standards Codification 815, Derivatives and Hedging ASC 815 , which establishes accounting and reporting standards for derivative instruments, including certain derivative instruments embedded in other contracts, and for hedging activities. They require that an entity recognize all derivatives as either assets or liabilities in the balance sheet and measure those instruments at fair value. If certain conditions are met, a derivative may be specifically designated as a hedge, the objective of which is to match the timing of gain or loss recognition on the hedging derivative with the recognition of (i) the changes in the fair value of the hedged asset or liability that are attributable to the hedged risk or (ii) the earnings effect of the hedged forecasted transaction. For a derivative not designated as a hedging instrument, the gain or loss is recognized in income in the period of change. Less: Allowance for doubtful accounts Accrued interest Total Less allowance for doubtful accounts Notes receivable, net While the failed Continuity Logic merger was pending, the Company made various bridge loans to Continuity Logic for working capital purposes, of which a total balance including accrued interest of was still and in default as of March 31, 2021, with a secured loan balance of . The loans matured on August 31, 2019, bear interest at per annum, and a portion of the notes for these loans are secured by a first priority perfected security interest on the accounts receivable of Continuity Logic, pursuant to a security agreement dated November 7, 2018. On February 4, 2019, the merger with Continuity Logic was terminated. The termination of the merger did not change of affect the continuing obligation of Continuity Logic to satisfy the future payment of these loans to the Company, and in 2019 the Company commenced a complaint against Continuity Logic LLC regarding the unpaid balance of these Notes Receivable. Continuity Logic is not a related party to the Company. The court action is ongoing while we also pursue reasonable collection activities. Due to the uncertainty of obtaining payment of these Notes, however, the Company has created an allowance for doubtful accounts of the entire amount of this Continuity Logic receivable, resulting no balance being recorded in our outstanding Notes Receivable for this debt. Additionally, the Company as a Petitioning Creditor has participated in an involuntary bankruptcy petition for Continuity Logic LLC in the United States Bankruptcy Court, District of New Jersey. Since the Company is the only secured lender, as a first priority and perfected lienholder, the Company expected the first of collected proceeds less any bankruptcy court related costs to be released as of the date of the settlement with Continuity Logic LLC. Termination of Continuity Logic Merger and Subsequent Settlement - In late 2018 the Company entered into a Merger Agreement with Continuity Logic, L.L.C., a New Jersey limited liability company (the Continuity Logic Merger to effect a merger which would have resulted in our shareholders as a group and the equity holders of Continuity Logic as a group each owning 50 of the post-merger combined companies. A key provision of this Continuity Logic Merger provided that the parties and their representatives were required to raise enough working capital to support the post-merger funding requirements of the combined companies, and also that the merger be completed by December 31, 2018. Accordingly, in February 2019 Continuity Logic terminated this merger since it was not consummated by December 31, 2018. The Company no longer has any relationship or involvement with the business or future prospects of Continuity Logic. F-11 Table of Contents has been long overdue and is still outstanding. These loans matured on , bear interest at per annum, and a substantial portion of the Notes is secured by a first-priority perfected security interest on the accounts receivable of Continuity Logic. The termination of the Continuity Logic Merger did not change or affect the continuing obligation of Continuity Logic to satisfy the payment of these loans to the Company. The Company took a reserve as of December 31, 2019 of the entire plus accrued interest against the balance of these notes receivables. Under a court approved settlement agreement dated June 28, 2021, which Continuity Logic LLC has satisfied its obligations within the time periods specified in the settlement agreement and made payments totaling for the year ended December 31, 2021, all of which was received in the third quarter. The Company recorded a gain on this settlement of , net of bad debt write offs recorded prior to the settlement, for the year ended December 31, 2021. Continuity Logic LLC was also required to pay the balance of the Company s unsecured claim plus additional accrued interested by April 30, 2022. Continuity Logic LLC satisfied all the required obligations of the settlement agreement by making payments totaling in the second quarter of 2022. Subsequently, both companies entered into a mutual release. During the year ended December 31, 2021, we settled and satisfied the complaint against us in a Minnesota District Court regarding our debt to Decathlon Alpha L.P. We used of these Continuity Logic LLC settlement proceeds to make an initial payment on a this settled complaint with Decathlon Alpha L.P. The Company made the final settlement payment of on September 15, 2022. Furniture Fixtures Construction in Process Less: Accumulated Depreciation Net Property and Equipment Construction in process include architect fees and other costs related to the surgery center in Ft. Myers, Florida. For the year ending December 31, 2021, we recorded of depreciation expense. F-12 Table of Contents Developed technology Scoreinc.com acquisition Permits Scoreinc.com acquisition Website Scoreinc.com acquisition Partner agreement Scoreinc.com acquisition Trade name Scoreinc.com acquisition Less: Accumulated Amortization Net Intangible assets Amortization expense for the year ended December 31, 2021 was . Our intangible assets will be amortized over three to five years and will be fully amortized at various times from 2024 to 2026. Amortization expense is estimated to be as follows: 2023 2024 2025 2026 A summary of changes in the Company s goodwill consists of the following during the twelve months ending December 31, 2021 and 2020 respectively: Goodwill customer contracts Less: Impairment Expense Goodwill Impairment Testing of Goodwill Goodwill is tested annually for impairment during the fourth quarter or earlier upon the occurrence of certain events or substantive changes in circumstances that indicate goodwill is more likely than not impaired. In connection with the annual goodwill impairment analysis performed during the fourth quarter of 2021, the Company determined that the fair value of our goodwill was greater than our book value, and therefore no goodwill impairment charge was recorded in 2021. In connection with the annual goodwill impairment analysis performed during the fourth quarter of 2020, the Company determined that the fair value of our goodwill on customer contracts was slightly less than our book value, and therefore we took a goodwill impairment charge in the fourth quarter of 2020, as some of our customers did not renew our software licensing and hosing contracts at the end of fiscal year 2020. , net of debt discount of . Such notes consist of short-term related party notes plus accrued interest of , long-term convertible related party notes plus accrued interest of , short-term non-related party notes payable of , and long-term non-related party convertible notes payable of . At December 31, 2020, we were indebted under various convertible and non-convertible Notes payable net of short-term and long-term debt discount of for a total amount of , consisting of short-term related party notes plus accrued interest of , long-term convertible related party notes plus accrued interest of , short term non-related party notes of and long term non-related party notes of , excluding the SBA/PPP loan. The short-term non-related party notes of consist of convertible notes of and of non-convertible notes payable. As of December 31, 2021 and 2020, most short-term notes are in default. F-13 Table of Contents convertible loan with Power Up Lending Group, This loan was paid back at the end of April 2020, less than three months later. The note had a interest rate, per annum, with prepayment penalty provisions within the first six months. In April 2020, we obtained a Small Business Administration SBA long-term loan under the federal COVID-19 Payroll Protection Program PPP for , administered by Richfield/Bloomington Credit Union, having a term of 24 months and bearing interest of per annum, with accrued interest at December 31, 2020. The portion of the loan proceeds used for labor, utilities and office costs may be subject to loan forgiveness under the terms of the SBA/PPP program. The balance as of December 31, 2020 of is included in the notes payable on the balance sheet. The SBA forgave the entire loan including accrued interest during 2021 resulting in the recognition of total loan forgiveness of for the year ended December 31, 2021, which is recorded as Other Income in the accompanying Statement of Operations. During July-August 2020, related parties converted outstanding accrued expenses into some of the notes described above. Michael Brown, a director and our former CEO, converted accrued salaries and funds put into the Company into new convertible Notes of maturing on . These convertible notes were subsequently converted into common stock before maturity. Capital Market Solutions, LLC converted its of accrued and unpaid consulting fees into a new short-term convertible note, simple interest rate, maturing on , for a total of . (See Notes 9 and 10). During July 2020, Garry Lowenthal, our former Chief Financial Officer and former Director, who resigned both positions on February 28, 2019, converted outstanding accrued expenses, including unpaid severance pay, pursuant to an Employment Agreement, into a note for . Another creditor, Rubicon Software Ltd, our outsourced development and technology consulting vendor converted of outstanding invoices into a convertible note of . . In January 2021, the Company issued a note payable to an accredited investors accruing interest at . The holder of the note has the option to convert the note into common stock currently held by CMS. In accordance with the acquisition of Score, as described below in Note 11, the Company issued the owner of Score a senior secured convertible note payable accruing no interest and convertible into not more than million shares of common stock at the higher of USD per share or the VWAP over the last 10 trading days prior to conversion. The Company also committed to issue the owner of Score Inc. an additional unsecured convertible note payable by March 31, 2023 equal to Score s average gross revenue during the calendar years ending 2021 and 2022. The unsecured note is convertible into not more than 10 million shares of common stock of Fision at USD per share. The company has accounted for this contingent liability at fair value and recorded it as Contingent liability Scorein.com acquisition on the balance sheet and the fair value conversion feature as derivative (See Note 3). The balance of this contingent note payable is as of December 31, 2021. The Company recorded a loss of as Other Expenses in the accompanying Statement of Operations to reflect the change in fair value of this contingency for the year ended December 31, 2021. Also in accordance with the acquisition of Score, the company assumed Score s outstanding PPP loan for . This loan was fully forgiven by the SBA in June 2022. On December 9, 2021, the Company issued a total of of convertible notes to two accredited investors bearing interest at and due on December 9, 2023. . shares of common stock and shares of preferred stock, both having .0001 par value per share. At December 31, 2021 and 2020 there were and outstanding shares of common stock and no outstanding shares of preferred stock. As of December 31, 2021, the Company has committed to issue shares of common stock including shares currently outstanding and shares committed through debt conversion features and warrants, which is shares over the authorized limit. These commitments include out of the money warrants. Common Shares Issued in 2021 Director services During the first quarter ended March 31, 2021, we issued a total of restricted common shares valued at to our directors for their quarterly management services, including shares valued at to director John Bode, shares valued at to director Michael Brown, shares valued at to director William Gerhauser, and shares valued at to director Greg Nagel. During the second quarter ended June 30, 2021, we issued a total of restricted common shares valued at to our directors for their quarterly management services, including shares valued at to John Bode, share valued at to Michael Brown, shares valued at to William Gerhauser, and shares valued at to Greg Nagel. During the third quarter ended September 30, 2021, we issued a total of restricted common shares valued at to our directors for their quarterly management services, including shares valued at to John Bode, shares valued at to Michael Brown, shares valued at to William Gerhauser, shares valued at to Greg Nagel, and shares valued at to Joshua Carmona. F-14 Table of Contents restricted common shares valued at to our directors for their quarterly management services, including shares valued at to John Bode, shares valued at to Joshua Carmona, shares valued at to William Gerhauser, shares valued at to Greg Nagel, and shares valued at to Michael Brown. Shares issued for note conversions From time to time, accredited investors holding company notes convert outstanding convertible notes payable to restricted common stock at conversion prices based on specific terms contained in each applicable note. Non-related party accredited investors converted of notes payable into shares of restricted common stock at an average conversion price of for the year ended December 31, 2020. For the year ended December 31, 2021, related parties converted debt and accrued interest into restricted common stock as follows: i) director Michael Brown converting his Note balance of at .05 per share into restricted shares; ii) affiliate Capital Market Solutions LLC (controlled by CEO and director William Gerhauser) converting its Note balances of at .05 per share into restricted shares; iii) affiliate Ignition Capital (also controlled by Mr. Gerhauser) converting its Note balance of at .03 per share into restricted shares; and iv) affiliate MGA Holdings (also controlled by Mr. Gerhauser) converting its Note balance of at .03 per share into restricted shares. Exercise of warrants In April 2021, two warrant holders who are accredited investors exercised their outstanding Common Stock Purchase Warrants at an exercise price of .025 per common share, thereby purchasing a total of restricted common shares for total proceeds to us of . Common Shares Issued in 2020 Director and advisory services The Company issued John Bode, an independent director, restricted common shares valued at for three months service in February 2020, and restricted common shares valued at for three months service in June 2020. Shares were calculated based on the closing stock price for each quarterly period of service. From August to December 2020 we issued the following restricted common shares for services as a director pursuant to independent director agreements. Shares were calculated based on the closing stock price for each quarterly period of service. i) shares valued at to John Bode for three months services from June-August 2020; ii) shares valued at to Gregory Nagel for six months services from April-September 2020; iii) shares valued at to John Bode for three months services from September-November 2020; and iv) shares valued at to Gregory Nagel for three months services from October-December 2020. In May 2020, we issued restricted common shares valued at to two accredited investors for advisory services to the Company. The shares were valued based on the closing stock price on date of transaction. True-up and penalty shares issued . Stock sold for cash In August 2020, incident to a private placement transaction, we received proceeds of from an individual private accredited investor, for which we issued restricted common shares along with a stock purchase warrant to purchase an additional common shares exercisable at .10 per share anytime during its two-year term. Shares issued for note conversions From time to time, accredited investors holding company notes convert outstanding convertible notes payable to restricted common stock at conversion prices based on specific terms contained in each applicable note. Non-related party accredited investors converted of notes payable into shares of restricted common stock at an average conversion price of for the year ended December 31, 2020. In December 2020, director Michael Brown converted of outstanding notes payable at per share to restricted common stock and affiliate CMS (controlled by director and CEO William Gerhauser) converted of outstanding notes payable at per share to restricted shares of common stock. as of December 31, 2020, which have been in default since 2019. Also, an additional short-term note for with affiliate MGA Capital LLC and two additional short-term notes for an aggregate of with affiliate Ignition Capital, LLC are also in default, as of December 31, 2020, whereby MGA Capital and Ignition Capital are partially owned by our director William Gerhauser. CMS, MGA Capital, and Ignition Capital all converted these short-term notes payable into common stock during 2021 as noted above. Incident to the note conversions during 2021 described above, certain related party noteholders converted an aggregate of of their Notes and accrued interest into shares of our restricted common stock. See Note 9 above. Incident to the note conversions during 2020 described above, certain related party noteholders converted an aggregate of of their Notes and accrued interest into shares of our restricted common stock. See Note 9 above. During the year ended December 31, 2021 and 2020, we issued a total of and shares of our restricted common stock to our directors for management services. See Note 9 above. F-15 Table of Contents of the membership interests of two Florida limited liability companies from Capital Market Solutions, LLC CMS ), which are Ft Myers ASC LLC Ft Myers ASC and ASC SoftDev LLC SoftDev ). These two LLCs were organized by CMS in the fall of 2020 to engage in the development and operation of medical Ambulatory Surgery Center. CMS is an affiliate of the Company and its largest shareholder, and is controlled by William Gerhauser, a director of the Company. This acquisition and its terms were specifically considered and approved by our two independent and disinterested directors, who also were advised by independent outside legal counsel. Neither Mr. Gerhauser nor any other representative of CMS participated in the vote of our Board of Directors to approve this acquisition. As of December 31, 2020, the Ft Myers ASC and SoftDev had no assets or liabilities, and accordingly, the Company has not recorded any assets or liabilities for this transaction. The financial position as of December 31, 2021 and results of operations for the year ended December 31, 2021 of Ft Myers ASC and SoftDev has been consolidated into the Company s financial statements. On April 1, 2021, the Company entered into a Memorandum of Understanding MOU with Scoreinc.com, Inc., a Puerto Rico Corporation Score and Joshua Carmona Carmona ), an individual who owned of Score. This MOU contained the material terms of the acquisition by Fision Corporation of 100 of Score including its subsidiaries. On May 30, 2021, the Company entered into a definitive Purchase and Sale Agreement PSA with Score, Carmona, and VIP Solutions, LLC VIP ), a subsidiary of Score, and pursuant to the PSA, the Company acquired 100 of Score to become a wholly owned subsidiary of Fision Corporation. The Company also acquired certain assets of VIP listed in the PSA. Score is an Act 73 company under Puerto Rico law that is in the enterprise software space and currently provides business to business solutions for approximately 100 US companies in the credit repair sector. Mr. Carmona owned 100 of Score capital stock free and clear of all liens and encumbrances of any kind, and VIP owned its assets acquired by the Company free and clear of all liens and encumbrances of any kind. For accounting and general purposes, the date of acquisition of Score was considered to be the May 30, 2021 date of the PSA, although the final closing occurred in August 2021 only after a required certified audit of Score s business operations and financial position was completed and accepted by Fision. In consideration for this acquisition of Score, we issued to Carmona a Senior Secured Promissory Note for substantially in the form attached as Exhibit 1 of the PSA, convertible into not more than ten ) million shares of common stock of Fision at the higher of USD per share or at the volume weighted average price (VWAP) over the last 10 trading days prior to conversion. Fision will also issue to Carmona not later than March 31, 2023 a second, unsecured promissory note in a form satisfactory to Fision and Carmona in an amount equal to Score s average gross revenue during calendar years ending 2021 and 2022, which will be convertible into not more than million shares of common stock of Fision at USD 0.20 per share and will contain the usual and customary protections and adjustments for future corporate actions, including but not limited to pricing adjustments for reverse stock splits. The Company is accounting for this contingency as a derivative (See Note 3). We also appointed Carmona as a member of our Board of Directors and as our Chief Operating Officer. The acquisition was accounted for using the purchase method of accounting. Score is a leading industry provider of software and services that empower credit repair organizations to be compliant and successful. ScoreCEO, its credit repair software as a service (SaaS) platform is used by many credit repair organizations to achieve success. The purchase of Score was a stock purchase, which included the acquisition of all its assets and the assumption of certain liabilities. The aggregate purchase price consisted of the following: Contingent consideration liability Additional consideration Purchase price The following table summarizes the estimated fair values of Score assets acquired and certain liabilities assumed at the date of acquisition to allocated the purchase price: Accounts receivable, net Intangible asset Developed technology Intangible asset Permits Intangible asset Website Intangible asset Partner agreement Intangible asset Trade name Goodwill Total Assets Accounts payable and accrued expenses (limited to 50,000) PPP loan payable Total Liabilities Net Purchase Price The Company has reported of revenue from Score in the Consolidated Statements of Operations for the year ended December 31, 2021 representing Score revenue for the period subsequent to acquisition. F-16 Table of Contents Costs of Sales Gross Profit Operating Expenses Sales and marketing Research and development General and administrative Operating Loss Other Income/(Expenses) Net Income/(Loss) million shares of common stock of Fision at USD per share and will contain the usual and customary protections and adjustments for future corporate actions, including but not limited to pricing adjustments for reverse stock splits. The Company has recorded this contingent liability as a Contingent liability Scoreinc.com acquisition in the balance sheet at an estimate fair value of as of December 31, 2021. We currently are not a party to any material legal proceedings against us, nor are we aware of any pending or threatened litigation that could have a material adverse effect on our business, operating results or financial condition. and for Federal and state purposes, respectively. . Given our history of net operating losses, our management has determined that it is more likely than not that we will not be able to realize the tax benefit of the carryforwards. Accordingly, we have not recognized a deferred tax asset for this benefit. Effective January 1, 2007, we adopted FASB guidelines that address the determination of whether tax benefits claimed or expected to be claimed on a tax return should be recorded in the financial statements. Under this guidance, we may recognize the tax benefit from an uncertain tax position only if it is more likely than not that the tax position will be sustained on examination by the taxing authorities, based on the technical merits of the position. The tax benefits recognized in the financial statements from such a position should be measured based on the largest benefit that has a greater than fifty percent likelihood of being realized upon ultimate settlement. This guidance also provides guidance on de-recognition, classification, interest and penalties on income taxes, accounting in interim periods and requires increased disclosures. At the date of adoption, and as of December 31, 2020 and 2019, we did not have a liability for unrecognized tax benefits, and no adjustment was required at adoption. Our policy is to record interest and penalties on uncertain tax provisions as income tax expense. As of December 31, 2021 and 2020, we have not accrued interest or penalties related to uncertain tax positions. Additionally, tax years 2018 through 2020 remain open to examination by the major taxing jurisdictions to which we are subject. Upon any attainment of sustainable taxable income by us, our management will assess the likelihood of realizing the tax benefit associated with the use of the carryforwards and will recognize a deferred tax asset at that time. F-17 Table of Contents .0 .0 Effects of permanent differences Effect of temporary differences Effects of state taxes (net of federal taxes) Change in valuation allowance Deferred income taxes reflect the net tax effects of temporary differences between the carrying amount of assets and liabilities for financial reporting purposes and the amounts used for income tax purposes. As of December 31, 2021 and 2020 our deferred income tax asset is calculated using nondeductible accrued interest of and , a nondeductible reserve for the notes receivable of and , respectively, and a cumulative estimated net tax operating loss of and , respectively, that is available to offset future taxable income, if any, in future periods, subject to expiration and other limitations imposed by the Internal Revenue Service. Management has considered our operating losses incurred to date and believes that a full valuation allowance against the deferred tax assets is required as of December 31, 2021 and 2020. Utilization of our net operating losses may be subject to substantial limitations if the Company experiences a 50 change in ownership, as provided by the Internal Revenue Code and similar state provisions. The Federal carryforward for the Net Operating Loss (NOL) for 2014-2017 expires in 2038, while the Federal carryforward NOL for 2018, 2019, 2020 and 2021 has no expiration date, and the state carryforward expires in 2023. Granted Forfeited or cancelled Balance December 31, 2020 Granted - Exercised Expired or cancelled Balance December 31, 2021 of our revenues, including from one customer and from another customer. For the year ended December 31, 2020 three customers each accounted for more than of our revenues, including from one customer, from a second customer and from a third customer. A significant reduction for any reason in the use of our software solutions by one or more of our major customers could harm our business materially. , administered by Richfield/Bloomington Credit Union, having a term of 24 months and bearing interest of per annum. The loan proceeds used for labor, office costs and utilities were subject to loan forgiveness under the terms of the SBA/PPP program. During the second quarter ended June 30, 2021, the SBA and Richfield/Bloomington Credit Union forgave any repayment of the entire loan and related accrued interest pursuant to which we recorded a total loan forgiveness of as Other Income. We assumed an SBA COVID-19 PPP loan of as part of our acquisition of Scoreinc.com. This loan was forgiven by the SBA in June 2022. shares of common stock to directors for their services during the period January 1, 2022 through the date of this report. The Company issued a director a on demand convertible note payable. The note is convertible at the greater of per share or the VWAP of the ten trading days prior to conversion. This director lent the Company an additional on October 3, 2022 for operating cash flow purposes. F-18 Table of Contents EXHIBITS Exhibit No. Description 2.1 Agreement and Plan of Merger, by and among the registrant, DE6 Newco Inc., a Minnesota corporation, and Fision Holdings, Inc., a Minnesota corporation (incorporated by reference to Exhibit 2.1 to the registrant s Current Report on Form 8-K filed on December 10, 2015) 2.2 First Amended and Restated Agreement and Plan of Merger dated December 21, 2018, by and among Fision Corporation and Continuity Logic, L.L.C. (incorporated by reference to the registrant s Current Report on Form 8-K filed on December 26, 2018) 2.3 Consulting Agreement, by and among the registrant, FISION Corporation, a Delaware corporation, and Capital Market Solutions, LLC., a Delaware limited liability corporation (filed with 2018 10-K Report) 3.1 Articles of Incorporation (incorporated by reference to Exhibit 3.1 of registrant s Form 10 filed on 4/6/2010) 3.2 Bylaws (incorporated by reference to Exhibit 3.2 of registrant s Form 10 filed on 4/6/2010) 10.1 Amended Director Agreement with John Bode, dated September 18, 2022 (filed herewith) 10.2 Independent Director Agreement with Greg Nagel, dated January 1, 2022 (filed herewith) 10.3 2011 Stock Option and Compensation Plan, as amended 12/30/2014 (incorporated by reference to the registrant s Current Report on Form 8-K filed on January 4, 2016) 10.4 2016 Equity Incentive Plan (incorporated by reference to the registrant s Registration Statement on Form S-8 filed on March 30, 2016) 10.5 Agreement to Acquire Two Florida LLCs for Ambulatory Surgery Center Business (incorporated by reference to Registrant s Current Report on Form 8-K filed with SEC on November 19, 2020) 31.1 Certification of CEO pursuant to Securities Exchange Act (filed herewith) 31.2 Certification of CFO pursuant to Securities Exchange Act (filed herewith) 32.1 Certification of CEO and CFO pursuant to 18 U.S.C. Section 1350 (filed herewith) 101 Interactive data files pursuant to Rule 405 of Regulation S-T 34 Table of Contents SIGNATURES Pursuant to the requirements of the Securities Exchange Act of 1934 , the registrant has duly caused this report to be signed on its behalf by the undersigned, thereunto duly authorized. FISION Corporation Dated: December 30, 2022 By: /s/ John Bode John Bode Interim Chief Executive Officer Pursuant to the requirements of the Securities Exchange Act of 1934, this report has been signed below by the following persons on behalf of the registrant and in the capacities and on the dates indicated. Dated: December 30, 2022 By: /s/ John Bode John Bode Director, Interim Chief Executive Officer (principal executiveofficer) Dated: December 30, 2022 By: /s/ John Bode John Bode Director, Interim Chief Executive Officer (principal financial and accounting officer) 35 

<EX-10.1>
 2
 fssn_ex101.htm
 AGREEMENT
 
 fssn_ex101.htm EXHIBIT 10.1 AMENDED DIRECTOR AGREEMENT This AMENDED DIRECTOR AGREEMENT (the "Agreement") is dated September 18, 2022 by and between FISION CORPORATION, a Delaware corporation (the "Company"), and JOHN BODE, an individual resident of the State of Florida (the "Director"). This Agreement amends the prior agreement between Mr. Bode and the Company dated March 1, 2022 as it relates to him serving as an independent Director of the Company. WHEREAS, the Mr. Bode has served as a Director or the Company since March 1, 2018 and the Company desires to enter into an agreement with the Director with respect to such appointment as Director and interim Chief Executive Officer for the period September 18, 2022 through December 31, 2022; and WHEREAS, the Director is willing continue to serve the Company on the terms set forth herein and in accordance with the provisions of this Agreement. NOW, THEREFORE, in consideration of the mutual covenants contained herein, the parties hereto agree as follows: 1. Position. Subject to the terms and provisions of this Agreement, the Company has caused the Director to be appointed, and the Director hereby agrees to serve the Company as its Interim Chief Executive Officer upon the terms and conditions hereinafter set forth, provided, however, that the Director's continued service on the Board of Directors of the Company (the "Board") and. 2. Duties. (a) During the Term (as defined herein), the Director make reasonable business efforts to attend all Board meetings and quarterly pre-scheduled Board and Committee meetings of the Board, and Management conference calls, serve on appropriate subcommittees as reasonably requested and agreed upon by the Board, make himself available to the Board at mutually convenient times and places, attend external meetings and presentations when agreed on in advance, as appropriate and convenient and perform such duties, services and responsibilities, and have the authority commensurate to such position. (b) The Director will use his best efforts to promote the interests of the Company. The Company recognizes that the Director (i) is or may perform work for another entity and (ii) sits or may sit on the board of directors of other entities, subject to any limitations set forth by the Sarbanes-Oxley Act of 2002 and limitations provided by any exchange or quotation service on which the Company's common stock is listed or traded. Notwithstanding the same, the Director will provide the Company with prior written notice of any future commitments to such entities and use reasonable business efforts to coordinate his respective commitments so as to fulfill his obligations to the Company and, in any event, will fulfill his legal obligations as a Director and interim Chief Executive Officer. Other than as set forth above, the Director will not, without the prior notification to the Board, engage in any other business activity which could materially interfere with the performance of his duties, services and responsibilities hereunder or which is in violation of the reasonable policies established from time to time by the Company, provided that the foregoing shall in no way limit his activities on behalf of (i) any current employer and its affiliates or (ii) the board of directors of any entities on which he currently sits. At such time as the Board receives such notification, the Board may require the resignation of the Director if it determines that such business activity does in fact materially interfere with the performance of the Director's duties, services and responsibilities hereunder. 3. Compensation. (a) Stock. For the four month period from September 1 to December 31, the Director shall receive 12,500 in the form of either restricted common stock of the Company calculated at the greater of the closing stock price on each of the quarterly anniversary dates listed or 0.05; or, preferred stock of the Company based on a 1 par value. The decision to provide the compensation as Restricted Common Stock or Preferred Stock shall be at the Company s option. Notwithstanding the foregoing, if the Director is no longer a member of the Board at quarterly grant dates for any reason (such as resignation, withdrawal, death, disability or any other reason), then any non-issued shares shall be irrefutably forfeited. Furthermore, the Director agrees that the shares shall be subject to any reasonable restrictions as determined by the Board of Directors of the Company related to the sale or transfer of those shares. (b) Independent Contractor. The Director's status during the Directorship Term shall be that of an independent contractor and not for any purpose, that of an employee or agent with authority to bind the Company in any respect. All payments and other consideration made or provided to the Director under this Section 3 shall be made or provided without withholding or deduction of any kind, and the Director shall assume sole responsibility for discharging all tax or other obligations associated therewith. Page 1 of 4 (c) Expense Reimbursements. During the Directorship Term, the Company shall reimburse the Director for all reasonable out-of-pocket expenses incurred by the Director in attending any in-person meetings, provided that the Director complies with the generally applicable policies, practices and procedures of the Company for submission of expense reports, receipts or similar documentation of such expenses. 4. Directorship Term. The "Directorship Term" as used in this Agreement shall mean the period commencing on the January 1, 2022 and terminating on the earliest of the following to occur: (a) the death of the Director; (b) the termination of the Director from his membership on the Board by the mutual agreement of the Company and the Director; (c) the removal of the Director from the Board by the majority stockholders of the Company; and (d) the resignation by the Director from the Board. 5. Director's Representation and Acknowledgment. The Director represents to the Company that his execution and performance of this Agreement shall not be in violation of any agreement or obligation (whether or not written) that he may have with or to any person or entity, including without limitation, any prior or current employer. The Director hereby acknowledges and agrees that this Agreement (and any other agreement or obligation referred to herein) shall be an obligation solely of the Company, and the Director shall have no recourse whatsoever against any stockholder of the Company or any of their respective affiliates with regard to this Agreement. 6. Directorship Covenants. (a) Unauthorized Disclosure. The Director agrees and understands that in the Director's position with the Company, the Director has been and will be exposed to and receive information relating to the confidential affairs of the Company, including, but not limited to, technical information, business and marketing plans, strategies, customer information, other information concerning the Company's products, promotions, development, financing, expansion plans, business policies and practices, and other forms of information considered by the Company to be confidential and in the nature of trade secrets. The Director agrees that during the Directorship Term and thereafter, the Director will keep such information confidential and will not disclose such information, either directly or indirectly, to any third person or entity without the prior written consent of the Company; provided, however, that (i) the Director shall have no such obligation to the extent such information is or becomes publicly known or generally known in the Company's industry other than as a result of the Director's breach of his obligations hereunder and (ii) the Director may, after giving prior notice to the Company to the extent practicable under the circumstances, disclose such information to the extent required by applicable laws or governmental regulations or judicial or regulatory process. This confidentiality covenant has no temporal, geographical or territorial restriction. Upon termination of the Directorship Term, the Director will promptly return to the Company and/or destroy at the Company's direction all property, keys, notes, memoranda, writings, lists, files, reports, customer lists, correspondence, tapes, disks, cards, surveys, maps, logs, machines, technical data, other product or document, and any summary or compilation of the foregoing, in whatever form, including, without limitation, in electronic form, which has been produced by, received by or otherwise submitted to the Director in the course or otherwise as a result of the Director's position with the Company during or prior to the Directorship Term, provided that the Company shall retain such materials and make them available to the Director if requested by him in connection with any litigation against the Director under circumstances in which (i) the Director demonstrates to the reasonable satisfaction of the Company that the materials are necessary to his defense in the litigation and (ii) the confidentiality of the materials is preserved to the reasonable satisfaction of the Company. (b) Non-Solicitation. During the Directorship Term and for a period of three (3) years thereafter, the Director shall not interfere with the Company's relationship with, or endeavor to entice away from the Company, any person who, on the date of shall not interfere with the termination of the Directorship Term and/or at any time during the one year period prior to the termination of the directorship Term, was an employee or customer of the Company or otherwise had a material business relationship with the Company. (c) Non-Compete. The Director agrees that during the Directorship Term and for a period of Three (3) years thereafter, he shall not in any manner, directly or indirectly, through any person, firm or corporation, alone or as a member of a partnership or as an officer, director, stockholder, investor or employee of or consultant to any other corporation or enterprise; engage in the business of developing, marketing, selling or supporting technology to or for businesses in which the Company engages in or in which the Company has an actual intention, as evidenced by the Company's written business plans, to engage in, within any geographic area in which the Company is then conducting such business. Nothing in this Section 6 shall prohibit the Director from being (i) a stockholder in a mutual fund or a diversified investment company or (ii) a passive owner of not more than three percent of the outstanding stock of any class of securities of a corporation, which are publicly traded, so long as the Director has no active participation in the business of such corporation. Page 2 of 4 (d) Insider Trading Guidelines. Director agrees to execute the Company's Insider Trading Guidelines in the form attached hereto. (e) Remedies. The Director agrees that any breach of the terms of this Section 6 would result in irreparable injury and damage to the Company for which the Company would have no adequate remedy at law; the Director therefore also agrees that in the event of said breach or any threat of breach, the Company shall be entitled to an immediate injunction and restraining order to prevent such breach and/or threatened breach and/or continued breach by the Director and/or any and all entities acting for and/or with the Director, without having to prove damages or paying a bond, in addition to any other remedies to which the Company may be entitled at law or in equity. The terms of this paragraph shall not prevent the Company from pursuing any other available remedies for any breach or threatened breach hereof, including, but not limited to, the recovery of damages from the Director. The Director acknowledges that the Company would not have entered into this Agreement had the Director not agreed to the provisions of this Section 6. (f) The provisions of this Section 6 shall survive any termination of the Directorship Term, and the existence of any claim or cause of action by the Director against the Company, whether predicated on this Agreement or otherwise, shall not constitute a defense to the enforcement by the Company of the covenants and agreements of this Section 6. 7. Indemnification. The Company agrees to indemnify the Director for his activities as a member of the Board to the fullest extent permitted under applicable law and shall use its best efforts to get Directors and Officers Insurance benefitting the Board, within one year of the Effective Date of this Agreement. 8. Non-Waiver of Rights. The failure to enforce at any time the provisions of this Agreement or to require at any time performance by the other party hereto of any of the provisions hereof shall in no way be construed to be a waiver of such provisions or to affect either the validity of this Agreement or any part hereof, or the right of either party hereto to enforce each and every provision in accordance with its terms. No waiver by either party hereto of any breach by the other party hereto of any provision of this Agreement to be performed by such other party shall be deemed a waiver of similar or dissimilar provisions at that time or at any prior or subsequent time. 9. Notices. Every notice relating to this Agreement shall be in writing and shall be given by personal delivery or by registered or certified mail, postage prepaid, return receipt requested; to: If to the Company: FISION Corporation Email: mike.brown@fisiononline.com If to the Director: John Bode 9086 The Lane Naples, Florida 34109 Email: jbode@eagleknoll.com Either of the parties hereto may change their address for purposes of notice hereunder by giving notice in writing to such other party pursuant to this Section 9. 10. Binding Effect/Assignment. This Agreement shall inure to the benefit of and be binding upon the parties hereto and their respective heirs, executors, personal representatives, estates, successors (including, without limitation, by way of merger) and assigns. Notwithstanding the provisions of the immediately preceding sentence, neither the Director nor the Company shall assign all or any portion of this Agreement without the prior written consent of the other party. Page 3 of 4 11. Entire Agreement. This Agreement (together with the other agreements referred to herein) sets forth the entire understanding of the parties hereto with respect to the subject matter hereof and supersedes all prior agreements, written or oral, between them as to such subject matter. 12. Severability. If any provision of this Agreement, or any application thereof to any circumstances, is invalid, in whole or in part, such provision or application shall to that extent be severable and shall not affect other provisions or applications of this Agreement. 13. Governing Law. This Agreement shall be governed by and construed in accordance with the laws of the State of Minnesota, without reference to the principles of conflict of laws. All actions and proceedings arising out of or relating to this Agreement shall be heard and determined in any court in Hennepin County, Minnesota and the parties hereto hereby consent to the jurisdiction of such courts in any such action or proceeding; provided, however, that neither party shall commence any such action or proceeding unless prior thereto the parties have in good faith attempted to resolve the claim, dispute or cause of action which is the subject of such action or proceeding through mediation by an independent third party. 14. Legal Fees. The parties hereto agree that the non-prevailing party in any dispute, claim, action or proceeding between the parties hereto arising out of or relating to the terms and conditions of this Agreement or any provision thereof (a "Dispute"), shall reimburse the prevailing party for reasonable attorney's fees and expenses incurred by the prevailing party in connection with such Dispute; provided, however, that the Director shall only be required to reimburse the Company for its fees and expenses incurred in connection with a Dispute if the Director's position in such Dispute was found by the court, arbitrator or other person or entity presiding over such Dispute to be frivolous or advanced not in good faith. 15. Modifications. Neither this Agreement nor any provision hereof may be modified, altered, amended or waived except by an instrument in writing duly signed by the party to be charged. 16. Tense and Headings. Whenever any words used herein are in the singular form, they shall be construed as though they were also used in the plural form in all cases where they would so apply. The headings contained herein are solely for the purposes of reference, are not part of this Agreement and shall not in any way affect the meaning or interpretation of this Agreement. 17. Counterparts. This Agreement may be executed in two or more counterparts, each of which shall be deemed to be an original but all of which together shall constitute one and the same instrument. IN WITNESS WHEREOF, the Company has caused this Independent Director Agreement to be executed by authority of its Board of Directors, and the Director has hereunto set his hand, on the day and year first above written. FISION CORPORATION DIRECTOR /s/ Michael Brown /s/ John Bode Michael Brown John Bode Page 4 of 4 

</EX-10.1>

<EX-10.2>
 3
 fssn_ex102.htm
 AGREEMENT
 
 fssn_ex102.htm EXHIBIT 10.2 INDEPENDENT DIRECTOR AGREEMENT This INDEPENDENT DIRECTOR AGREEMENT (the "Agreement") is dated January 1, 2022 by and between FISION CORPORATION, a Delaware corporation (the "Company"), and GREG NAGEL, an individual resident of the State of Texas (the "Director"). This Agreement amends the prior agreement between Mr. Nagel and the Company dated April 1, 2021 as it relates to the quarterly share compensation due for the period January 1, 2022 to March 31, 2022. WHEREAS, the Mr. Nagel has served as a Director or the Company since April 1, 2020 and the Company desires to enter into an agreement with the Director with respect to such appointment for the period from January 1, 2022 through December 31, 2022; and WHEREAS, the Director is willing continue to serve the Company on the terms set forth herein and in accordance with the provisions of this Agreement. NOW, THEREFORE, in consideration of the mutual covenants contained herein, the parties hereto agree as follows: 1. Position. Subject to the terms and provisions of this Agreement, the Company shall cause the Director to be appointed, and the Director hereby agrees to serve the Company in such position upon the terms and conditions hereinafter set forth, provided, however, that the Director's continued service on the Board of Directors of the Company (the "Board") after the initial one- year term on the Board shall be subject to any necessary approval by the Company's stockholders. Director is classified as an Independent Director as defined by NASDAQ Marketplace Rule 4200(a)(15) and also Section lOA of the Securities and Exchange Act of 1934 which specifies general criteria for independence. 2. Duties. (a) During the Directorship Term (as defined herein), the Director make reasonable business efforts to attend all Board meetings and quarterly pre-scheduled Board and Committee meetings of the Board, and Management conference calls, serve on appropriate subcommittees as reasonably requested and agreed upon by the Board, make himself available to the Company at mutually convenient times and places, attend external meetings and presentations when agreed on in advance, as appropriate and convenient, and perform such duties, services and responsibilities, and have the authority commensurate to such position. (b) The Director will use his best efforts to promote the interests of the Company. The Company recognizes that the Director (i) is or may become a full-time executive employee of another entity and that his responsibilities to such entity must have priority and (ii) sits or may sit on the board of directors of other entities, subject to any limitations set forth by the Sarbanes-Oxley Act of 2002 and limitations provided by any exchange or quotation service on which the Company's common stock is listed or traded. Notwithstanding the same, the Director will provide the Company with prior written notice of any future commitments to such entities and use reasonable business efforts to coordinate his respective commitments so as to fulfill his obligations to the Company and, in any event, will fulfill his legal obligations as a Director. Other than as set forth above, the Director will not, without the prior notification to the Board, engage in any other business activity which could materially interfere with the performance of his duties, services and responsibilities hereunder or which is in violation of the reasonable policies established from time to time by the Company, provided that the foregoing shall in no way limit his activities on behalf of (i) any current employer and its affiliates or (ii) the board of directors of any entities on which he currently sits. At such time as the Board receives such notification, the Board may require the resignation of the Director if it determines that such business activity does in fact materially interfere with the performance of the Director's duties, services and responsibilities hereunder. (c) Since the Director is to be classified as an independent director at the time of his appointment, the Director shall promptly inform the Board of any circumstances that would likely affect such independent status. The Director shall inform the Board within 10 business days of the Director's appointment of any held (indirect and indirect) personal interests which may conflict with the Company and its business. The Director will at all times act as a fiduciary in the service and best interests of the Company. In addition, the Director agrees to (i) provide all information regarding himself or herself as the Company requires to satisfy its disclosure obligations under applicable securities laws; (ii) timely file with the Securities and Exchange Commission all reports and schedules required of the Director in his or her personal capacity by virtue of his or her relationship with the Company (e.g. Forms 3, 4 and 5 as contemplated by Section 16(a) of the Securities Exchange Act of 1934). Page 1 of 4 3. Compensation. (a) Stock. On a quarterly basis, on March 31, June 30, September 30 and December 31, the Director shall receive 12,500 in the form of either restricted common stock of the Company calculated at the greater of the closing stock price on each of the quarterly anniversary dates listed or 0.05; or, preferred stock of the Company based on a 1 par value. The decision to provide the compensation as Restricted Common Stock or Preferred Stock shall be at the Company s option. Notwithstanding the foregoing, if the Director is no longer a member of the Board at quarterly grant dates for any reason (such as resignation, withdrawal, death, disability or any other reason), then any non-issued shares shall be irrefutably forfeited. Furthermore, the Director agrees that the shares shall be subject to any reasonable restrictions as determined by the Board of Directors of the Company related to the sale or transfer of those shares. (b) Independent Contractor. The Director's status during the Directorship Term shall be that of an independent contractor and not for any purpose, that of an employee or agent with authority to bind the Company in any respect. All payments and other consideration made or provided to the Director under this Section 3 shall be made or provided without withholding or deduction of any kind, and the Director shall assume sole responsibility for discharging all tax or other obligations associated therewith. (c) Expense Reimbursements. During the Directorship Term, the Company shall reimburse the Director for all reasonable out-of-pocket expenses incurred by the Director in attending any in-person meetings, provided that the Director complies with the generally applicable policies, practices and procedures of the Company for submission of expense reports, receipts or similar documentation of such expenses. 4. Directorship Term. The "Directorship Term," as used in this Agreement, shall mean the period commencing on the January 1, 2022 and terminating on the earliest of the following to occur: (a) the death of the Director; (b) the termination of the Director from his membership on the Board by the mutual agreement of the Company and the Director; (c) the removal of the Director from the Board by the majority stockholders of the Company; and (d) the resignation by the Director from the Board. 5. Director's Representation and Acknowledgment. The Director represents to the Company that his execution and performance of this Agreement shall not be in violation of any agreement or obligation (whether or not written) that he may have with or to any person or entity, including without limitation, any prior or current employer. The Director hereby acknowledges and agrees that this Agreement (and any other agreement or obligation referred to herein) shall be an obligation solely of the Company, and the Director shall have no recourse whatsoever against any stockholder of the Company or any of their respective affiliates with regard to this Agreement. 6. Directorship Covenants. (a) Unauthorized Disclosure. The Director agrees and understands that in the Director's position with the Company, the Director has been and will be exposed to and receive information relating to the confidential affairs of the Company, including, but not limited to, technical information, business and marketing plans, strategies, customer information, other information concerning the Company's products, promotions, development, financing, expansion plans, business policies and practices, and other forms of information considered by the Company to be confidential and in the nature of trade secrets. The Director agrees that during the Directorship Term and thereafter, the Director will keep such information confidential and will not disclose such information, either directly or indirectly, to any third person or entity without the prior written consent of the Company; provided, however, that (i) the Director shall have no such obligation to the extent such information is or becomes publicly known or generally known in the Company's industry other than as a result of the Director's breach of his obligations hereunder and (ii) the Director may, after giving prior notice to the Company to the extent practicable under the circumstances, disclose such information to the extent required by applicable laws or governmental regulations or judicial or regulatory process. This confidentiality covenant has no temporal, geographical or territorial restriction. Upon termination of the Directorship Term, the Director will promptly return to the Company and/or destroy at the Company's direction all property, keys, notes, memoranda, writings, lists, files, reports, customer lists, correspondence, tapes, disks, cards, surveys, maps, logs, machines, technical data, other product or document, and any summary or compilation of the foregoing, in whatever form, including, without limitation, in electronic form, which has been produced by, received by or otherwise submitted to the Director in the course or otherwise as a result of the Director's position with the Company during or prior to the Directorship Term, provided that the Company shall retain such materials and make them available to the Director if requested by him in connection with any litigation against the Director under circumstances in which (i) the Director demonstrates to the reasonable satisfaction of the Company that the materials are necessary to his defense in the litigation and (ii) the confidentiality of the materials is preserved to the reasonable satisfaction of the Company.(b) Non-Solicitation. During the Directorship Term and for a period of three (3) years thereafter, the Director shall not interfere with the Company's relationship with, or endeavor to entice away from the Company, any person who, on the date of shall not interfere with the termination of the Directorship Term and/or at any time during the one year period prior to the termination of the directorship Term, was an employee or customer of the Company or otherwise had a material business relationship with the Company. Page 2 of 4 (c) Non-Compete. The Director agrees that during the Directorship Term and for a period of Three (3) years thereafter, he shall not in any manner, directly or indirectly, through any person, firm or corporation, alone or as a member of a partnership or as an officer, director, stockholder, investor or employee of or consultant to any other corporation or enterprise; engage in the business of developing, marketing, selling or supporting technology to or for businesses in which the Company engages in or in which the Company has an actual intention, as evidenced by the Company's written business plans, to engage in, within any geographic area in which the Company is then conducting such business. Nothing in this Section 6 shall prohibit the Director from being (i) a stockholder in a mutual fund or a diversified investment company or (ii) a passive owner of not more than three percent of the outstanding stock of any class of securities of a corporation, which are publicly traded, so long as the Director has no active participation in the business of such corporation. (d) Insider Trading Guidelines. Director agrees to execute the Company's Insider Trading Guidelines in the form attached hereto. (e) Remedies. The Director agrees that any breach of the terms of this Section 6 would result in irreparable injury and damage to the Company for which the Company would have no adequate remedy at law; the Director therefore also agrees that in the event of said breach or any threat of breach, the Company shall be entitled to an immediate injunction and restraining order to prevent such breach and/or threatened breach and/or continued breach by the Director and/or any and all entities acting for and/or with the Director, without having to prove damages or paying a bond, in addition to any other remedies to which the Company may be entitled at law or in equity. The terms of this paragraph shall not prevent the Company from pursuing any other available remedies for any breach or threatened breach hereof, including, but not limited to, the recovery of damages from the Director. The Director acknowledges that the Company would not have entered into this Agreement had the Director not agreed to the provisions of this Section 6. (f) The provisions of this Section 6 shall survive any termination of the Directorship Term, and the existence of any claim or cause of action by the Director against the Company, whether predicated on this Agreement or otherwise, shall not constitute a defense to the enforcement by the Company of the covenants and agreements of this Section 6. 7. Indemnification. The Company agrees to indemnify the Director for his activities as a member of the Board to the fullest extent permitted under applicable law and shall use its best efforts to get Directors and Officers Insurance benefitting the Board, within one year of the Effective Date of this Agreement. 8. Non-Waiver of Rights. The failure to enforce at any time the provisions of this Agreement or to require at any time performance by the other party hereto of any of the provisions hereof shall in no way be construed to be a waiver of such provisions or to affect either the validity of this Agreement or any part hereof, or the right of either party hereto to enforce each and every provision in accordance with its terms. No waiver by either party hereto of any breach by the other party hereto of any provision of this Agreement to be performed by such other party shall be deemed a waiver of similar or dissimilar provisions at that time or at any prior or subsequent time. 9. Notices. Every notice relating to this Agreement shall be in writing and shall be given by personal delivery or by registered or certified mail, postage prepaid, return receipt requested; to: If to the Company: FISION Corporation Email: mike.brown@fisiononline.com If to the Director: Greg Nagel 7925 Rolling Acres Trail Fair Oaks Ranch, TX 78015 Email: grnags1@gmail.com Either of the parties hereto may change their address for purposes of notice hereunder by giving notice in writing to such other party pursuant to this Section 9. Page 3 of 4 10. Binding Effect/Assignment. This Agreement shall inure to the benefit of and be binding upon the parties hereto and their respective heirs, executors, personal representatives, estates, successors (including, without limitation, by way of merger) and assigns. Notwithstanding the provisions of the immediately preceding sentence, neither the Director nor the Company shall assign all or any portion of this Agreement without the prior written consent of the other party. 11. Entire Agreement. This Agreement (together with the other agreements referred to herein) sets forth the entire understanding of the parties hereto with respect to the subject matter hereof and supersedes all prior agreements, written or oral, between them as to such subject matter. 12. Severability. If any provision of this Agreement, or any application thereof to any circumstances, is invalid, in whole or in part, such provision or application shall to that extent be severable and shall not affect other provisions or applications of this Agreement. 13. Governing Law. This Agreement shall be governed by and construed in accordance with the laws of the State of Minnesota, without reference to the principles of conflict of laws. All actions and proceedings arising out of or relating to this Agreement shall be heard and determined in any court in Hennepin County, Minnesota and the parties hereto hereby consent to the jurisdiction of such courts in any such action or proceeding; provided, however, that neither party shall commence any such action or proceeding unless prior thereto the parties have in good faith attempted to resolve the claim, dispute or cause of action which is the subject of such action or proceeding through mediation by an independent third party. 14. Legal Fees. The parties hereto agree that the non-prevailing party in any dispute, claim, action or proceeding between the parties hereto arising out of or relating to the terms and conditions of this Agreement or any provision thereof (a "Dispute"), shall reimburse the prevailing party for reasonable attorney's fees and expenses incurred by the prevailing party in connection with such Dispute; provided, however, that the Director shall only be required to reimburse the Company for its fees and expenses incurred in connection with a Dispute if the Director's position in such Dispute was found by the court, arbitrator or other person or entity presiding over such Dispute to be frivolous or advanced not in good faith. 15. Modifications. Neither this Agreement nor any provision hereof may be modified, altered, amended or waived except by an instrument in writing duly signed by the party to be charged. 16. Tense and Headings. Whenever any words used herein are in the singular form, they shall be construed as though they were also used in the plural form in all cases where they would so apply. The headings contained herein are solely for the purposes of reference, are not part of this Agreement and shall not in any way affect the meaning or interpretation of this Agreement. 17. Counterparts. This Agreement may be executed in two or more counterparts, each of which shall be deemed to be an original but all of which together shall constitute one and the same instrument. IN WITNESS WHEREOF, the Company has caused this Independent Director Agreement to be executed by authority of its Board of Directors, and the Director has hereunto set his hand, on the day and year first above written. FISION CORPORATION DIRECTOR /s/ Michael Brown /s/ Greg Nagel Michael Brown Greg Nagel Page 4 of 4 

</EX-10.2>

<EX-31.1>
 4
 fssn_ex311.htm
 CERTIFICATION
 
 fssn_ex311.htm EXHIBIT 31.1 Rule 13a-14(a)/15d-14(a) Certification I, John Bode, certify that: 1. I have reviewed this annual report on Form 10-K for the year ended December 31, 2021 of FISION Corporation. 2. Based on my knowledge, this report does not contain any untrue statement of a material fact or omit to state a material fact necessary to make the statements made, in light of the circumstances under which such statements were made, not misleading with respect to the period covered by this report; 3. Based on my knowledge, the financial statements, and other financial information contained in this report, fairly present in all material respects the financial condition, results of operations and cash flows of the registrant as of, and for, the periods presented this report; 4. The registrant s other certifying officer(s) and I are responsible for establishing and maintaining disclosure controls and procedures (as defined in Exchange Act Rules 13a-15(e) and 15d-15(e)) and internal control over financial reporting (as defined in Exchange Act Rules 13a-15(f) and 15d-15(f)) for the registrant and have: (a) Designed such disclosure controls and procedures, or caused such disclosure controls and procedures to be designed under our supervision, to ensure that material information relating to the registrant, including its consolidated subsidiaries, is made known to us by others within those entities, particularly during the period in which this report is being prepared; (b) Designed such internal control over financial reporting, or caused such internal control over financial reporting to be designed under our supervision, to provide reasonable assurance regarding the reliability of financial reporting and the preparation of financial statements for external purposes in accordance with generally accepted accounting principles; (c) Evaluated the effectiveness of the registrant s disclosure controls and procedures and presented in this report our conclusions about the effectiveness of the disclosure controls and procedures, as of the end of the period covered by this report based on such evaluation; and (d) Disclosed in this report any change in the registrant s internal control over financial reporting that occurred during the registrant s most recent fiscal quarter (the registrant s fourth fiscal quarter in the case of an annual report) that has materially affected, or is reasonably likely to materially affect, the registrant s internal control over financial reporting; and 5. The registrant s other certifying officer(s) and I have disclosed, based on our most recent evaluation of internal control over financial reporting, to the registrant s auditors and the audit committee of the registrant s board of directors (or persons performing the equivalent functions): (a) All significant deficiencies and material weaknesses in the design or operation of internal control over financial reporting which are reasonably likely to adversely affect the registrant s ability to record, process, summarize and report financial information; and (b) Any fraud, whether or not material, that involves management or other employees who have a significant role in the registrant s internal control over financial reporting. Dated: December 30, 2022 By: /s/ John Bode John Bode Interim Chief Executive Officer, Principal executive officer 

</EX-31.1>

<EX-31.2>
 5
 fssn_ex312.htm
 CERTIFICATION
 
 fssn_ex312.htm EXHIBIT 31.2 Rule 13a-14(a)/15d-14(a) Certification I, John Bode, certify that: 1. I have reviewed this annual report on Form 10-K for the year ended December 31, 2021 of FISION Corporation. 2. Based on my knowledge, this report does not contain any untrue statement of a material fact or omit to state a material fact necessary to make the statements made, in light of the circumstances under which such statements were made, not misleading with respect to the period covered by this report; 3. Based on my knowledge, the financial statements, and other financial information contained in this report, fairly present in all material respects the financial condition, results of operations and cash flows of the registrant as of, and for, the periods presented this report; 4. The registrant s other certifying officer(s) and I are responsible for establishing and maintaining disclosure controls and procedures (as defined in Exchange Act Rules 13a-15(e) and 15d-15(e)) and internal control over financial reporting (as defined in Exchange Act Rules 13a-15(f) and 15d-15(f)) for the registrant and have: (a) Designed such disclosure controls and procedures, or caused such disclosure controls and procedures to be designed under our supervision, to ensure that material information relating to the registrant, including its consolidated subsidiaries, is made known to us by others within those entities, particularly during the period in which this report is being prepared; (b) Designed such internal control over financial reporting, or caused such internal control over financial reporting to be designed under our supervision, to provide reasonable assurance regarding the reliability of financial reporting and the preparation of financial statements for external purposes in accordance with generally accepted accounting principles; (c) Evaluated the effectiveness of the registrant s disclosure controls and procedures and presented in this report our conclusions about the effectiveness of the disclosure controls and procedures, as of the end of the period covered by this report based on such evaluation; and (d) Disclosed in this report any change in the registrant s internal control over financial reporting that occurred during the registrant s most recent fiscal quarter (the registrant s fourth fiscal quarter in the case of an annual report) that has materially affected, or is reasonably likely to materially affect, the registrant s internal control over financial reporting; and 5. The registrant s other certifying officer(s) and I have disclosed, based on our most recent evaluation of internal control over financial reporting, to the registrant s auditors and the audit committee of the registrant s board of directors (or persons performing the equivalent functions): (a) All significant deficiencies and material weaknesses in the design or operation of internal control over financial reporting which are reasonably likely to adversely affect the registrant s ability to record, process, summarize and report financial information; and (b) Any fraud, whether or not material, that involves management or other employees who have a significant role in the registrant s internal control over financial reporting. Dated: December 30, 2022 By: /s/ John Bode John Bode Interim Chief Executive Officer and Principal financial and accounting officer 

</EX-31.2>

<EX-32.1>
 6
 fssn_ex321.htm
 CERTIFICATION
 
 fssn_ex321.htm EXHIBIT 32.1 Section 1350 Certification In connection with the Annual Report of FISION Corporation, a Delaware corporation (the Company ), on Form 10-K for the year ended December 31, 2021 as filed with the Securities and Exchange Commission (the Report ), I, John Bode, Chief Executive Officer of the Company and I, John Bode, Principal Financial and Accounting Officer of the Company, do hereby certify, pursuant to 18 U.S.C. Sec. 1350, as adopted pursuant to Sec. 906 of the Sarbanes-Oxley Act of 2002, that : 1. The Report fully complies with the requirements of Section 13(a) or 15(d) of the Securities and Exchange Act of 1934, as amended, and 2. The information contained in the Report fairly presents, in all material respects, the financial conditions and results of operations of the Company. December 30, 2022 By: /s/ John Bode John Bode Interim Chief Executive Officer, Principal executive officer December 30, 2022 By: /s/ John Bode John Bode Interim Chief Executive Officer, Principal financial and accounting officer A signed original of this written statement required by Section 906, or other document authenticating, acknowledging or otherwise adopting the signature that appears in typed form within the electronic version of this written statement has been provided to the Company and will be retained by the Company and furnished to the Securities and Exchange Commission or its staff upon request. 

</EX-32.1>

<EX-101.SCH>
 7
 fssn-20211231.xsd
 XBRL TAXONOMY EXTENSION SCHEMA

</EX-101.LAB>

<EX-101.CAL>
 9
 fssn-20211231_cal.xml
 XBRL TAXONOMY EXTENSION CALCULATION LINKBASE

</EX-101.CAL>

<EX-101.PRE>
 10
 fssn-20211231_pre.xml
 XBRL TAXONOMY EXTENSION PRESENTATION LINKBASE

</EX-101.PRE>

<EX-101.DEF>
 11
 fssn-20211231_def.xml
 XBRL TAXONOMY EXTENSION DEFINITION LINKBASE

</EX-101.DEF>

"""

In [6]:
# Get the firm's CIK
cik_match = re.search(r'CENTRAL INDEX KEY:\s+(\d+)', content)
cik = cik_match.group(1) if cik_match else 'Unknown'

# Get the firm's name
company_name_match = re.search(r"COMPANY CONFORMED NAME:\s+([^\n]+)", content)
company_name = company_name_match.group(1) if company_name_match else 'Unknown'

# Get the date (Filed as of date)
date_match = re.search(r'FILED AS OF DATE:\s+(\d{8})', content)
date = date_match.group(1) if date_match else 'Unknown'

In [8]:
# Extract item text
item_1 = item_search(r"(Item 1)(.*?)(Item 1A)(.*?)", content)
item_1A = item_search(r"(Item 1A)(.*?)(Item 1B)", content)
item_7A = item_search(r"(Item 7a\.)(.*?)(Item 8\.)", content)
item_1A

'ITEM 1A RISK FACTORS An investment in our common stock is speculative and involves a high degree of risk. You should carefully consider the following risks, along with all of the other information contained in this Annual Report, before considering making an investment decision to purchase any of our securities. If one or more of the following risks occur, our business, financial condition and results of operations could suffer materially. In that case, the price of our common shares in any trading market could decline materially, and you could lose a substantial part or all of your investment. Risks Related to Our Business We have a limited operating history and substantial accumulated losses, and we anticipate incurring continued losses. Our business must be considered and evaluated in light of the risks, expenses, and problems frequently encountered by companies in their early stages of development and commercialization of products, and particularly companies like us participating 

In [9]:
# Clean the item text
item_1_cleaned = clean_text(item_1, keywords, sequence_length)
item_1A_cleaned = clean_text(item_1A, keywords, sequence_length)
item_7A_cleaned = clean_text(item_7A, keywords, sequence_length)
item_1A_cleaned

's We operate in a highly competitive industry, and we may not be able to compete successfully. Our market is highly competitive, with many companies providing solutions competitive to our software platform. Well-known and established competitors include Marketo (Adobe), Eloqua (Oracle), Unica (IBM), Hubspot, ExactTarget (Salesforce), Aprimo, Responsys and Silverpop. We expect additional material competitors to emerge in the future from time to time. Most of our current and potential competitors have significantly more personnel, financial, technical, marketing and other resources than we possess, and accordingly they are able to devote substantially greater resources than us to development, marketing, sales, and support of their products and services. We lack many things which most of our competitors have, including an established brand and name recognition, existing relationships with a large customer base, the ability to undertake costly and extensive marketing campaigns, and large 

In [10]:
print(f"We reduced the character length in Item 1A from {len(item_1A)} to {len(item_1A_cleaned)}")

We reduced the character length in Item 1A from 29060 to 10187
